# Intro

## What are agents?

![[image source: Tweet from Abhishek Thakur](https://x.com/abhi1thakur/status/1873697074405122144)](imgs/say_agentic_one_more_time.jpeg)

Let's start with some definitions of agents from different sources.


::: {.callout-note}
## Agent Definition from LangChain Blog Post - [source](https://blog.langchain.dev/what-is-an-agent/)
An AI agent is a system that uses an LLM to decide the control flow of an application.
:::

::: {.callout-note}
## Agent Definition from AWS - [source](https://aws.amazon.com/what-is/ai-agents/)
An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals.
:::

::: {.callout-note}
## Agent Definition from Chip Huyen's Book "AI Engineering" - [source](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

An **agent** is anything that can perceive its environment and act upon that environment. This means that an **agent** is characterized by the environment it operates in and the set of actions it can perform.
:::



::: {.callout-note}
## Agent Definition from Mongo DB Blog Post - [source](https://www.mongodb.com/resources/basics/artificial-intelligence/ai-agents#what-is-an-ai-agent)

An AI **agent** is a computational entity with an awareness of its environment that’s equipped with faculties that enable perception through input, action through tool use, and cognitive abilities through foundation models backed by long-term and short-term memory.
:::

::: {.callout-note}
## Agent Definition from Anthropic - [source](https://www.anthropic.com/research/building-effective-agents)

**"Agent"** can be defined in several ways. Some customers define agents as fully autonomous systems that operate independently over extended periods, using various tools to accomplish complex tasks. Others use the term to describe more prescriptive implementations that follow predefined workflows. At Anthropic, we categorize all these variations as **agentic** systems, but draw an important architectural distinction between **workflows** and **agents**:

**Workflows** are systems where LLMs and tools are orchestrated through predefined code paths.

**Agents**, on the other hand, are systems where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.
:::


::: {.callout-note}
## Agent Definition from Hugging Face Blog Post on `smolagents` - [source](https://huggingface.co/blog/smolagents)

Any efficient system using AI will need to provide LLMs some kind of access to the real world: for instance the possibility to call a search tool to get external information, or to act on certain programs in order to solve a task. In other words, LLMs should have agency. Agentic programs are the gateway to the outside world for LLMs.


**Agents** are programs where LLM outputs control the workflow. Note that with this definition, **"agent"** is not a discrete, 0 or 1 definition: instead, "agency" evolves on a continuous spectrum, as you give more or less power to the LLM on your workflow.
:::


## Is it an Agent? Is it Agentic? It's more like a spectrum with a lot of gray area!

![[image source: Tweet from Andrew Ng](https://x.com/AndrewYNg/status/1801295202788983136)](imgs/its_agentic.jpg)

There is a lot of debate and discussion on what exactly is an agent and what is not an agent. I think there is a lot of gray area here and something we 
have to just accept, at least for now. I think Andrew Ng makes some really good points in this [tweet](https://x.com/AndrewYNg/status/1801295202788983136). As Andrew points out, rather than engaging in binary debates about whether something qualifies as a "true agent," we should think about systems as existing on a spectrum of agent-like qualities.  The adjective "agentic" itself becomes particularly useful here, allowing us to describe systems that incorporate agent-like patterns to different degrees without getting caught in restrictive definitions.

This spectrum-based view is reinforced by Anthropic's recent [blog post](https://www.anthropic.com/research/building-effective-agents) on agents. They acknowledge that while they draw an architectural distinction between workflows (systems with predefined code paths) and agents (systems with dynamic control), they categorize both under the broader umbrella of "agentic systems." Similarly, we saw from one of our definitions above that "agent" isn't a discrete, 0 or 1 definition, but rather evolves on a continuous spectrum as you give more or less power to the LLM in your system. This aligns with Andrew Ng's observation that there's a gray zone between what clearly is not an agent (prompting a model once) and what clearly is (an autonomous system that plans, uses tools, and executes multiple steps independently).

![[image source: Blog post from Nathan Lambert on the AI Agent Spectrum](https://www.interconnects.ai/p/the-ai-agent-spectrum)](imgs/llm_loop_is_agent.webp)

Nathan Lambert also writes about the AI agent spectrum in this [blog post](https://www.interconnects.ai/p/the-ai-agent-spectrum). Nathan discusses that the simplest system on this spectrum would be any tool-use language model and that the spectrum of agents increases in complexity from there. I like how Nathan makes the point that the spectrum will continue to evolve and that the definition of an agent will continue to change as the field evolves. Over time, certain technologies will reach milestones where they become definitive examples of AI agents. Therefore at some point, basic tool use with an LLM may not be considered an agent, even though it's the basic starting point on the agentic spectrum.

![[image source: Tweet from Hamel Husain](https://x.com/HamelHusain/status/1867289397898752222)](imgs/llm_workflow_vs_loop.png)

Personally, agents and agentic workflows are still so new to me and I have a lot to learn on this topic. I have deployed LLMs in production as well as built some applications where LLMs use function calling (tools) within a conversational chat interface. So I think some of my previous work has fallen somewhere within this AI agentic spectrum, even if it's at one end of the spectrum. I'm going to  keep an open mind and avoid getting caught up in debates about categorical definitions. I'll try to avoid the hype and marketing fluff but be on the lookout for innovation and practical applications. 



# The Tool Calling Loop: A Building Block for Agentic Systems {#sec-tool_calling_loop}


![[image source: Tweet from Abhishek Thakur](https://x.com/abhi1thakur/status/1875159964785987904)](imgs/just_loops_meme.jpeg)

So where do we even start on this spectrum of AI agents? Practically, I think the first step is to start with an LLM equipped with tools.
I think this is what Anthropic refers to as the ["The augmented LLM"](https://www.anthropic.com/research/building-effective-agents).

![[image source: Blog post from Anthropic on Building effective agents](https://www.anthropic.com/research/building-effective-agents)](imgs/the_augmented_llm.png)

This is the building block, an LLM equipped with tools. I think we need to take it slightly further and make it clear we need a tool calling loop. The entire process is kicked off by sending a user request to the LLM. The LLM then decides on the initial tool calls to be made in the first step. These tool calls could be executed in parallel if they are independent of one another. After calling the initial tools, the LLM can choose whether to repeat follow up tool calls, which are dependent on the results of previous tool calls. Implementing this logic together within a loop is what I refer to as the "tool calling loop". 

I wrote about this tool calling loop a while ago in a previous [blog_post](https://drchrislevy.github.io/posts/anthropic/anthropic.html#my-own-wrapper-with-an-emphasis-on-the-tool-calling-loop). Here is an image I created at the time to illustrate the concept. 

![[image source: previous blog post](https://drchrislevy.github.io/posts/anthropic/anthropic.html#my-own-wrapper-with-an-emphasis-on-the-tool-calling-loop)](imgs/llm_tool_call_loop.png){height=900px}

One could call this tool calling loop "agentic" since the LLM is making decisions on what tool calls to make. Or maybe we just call it an "augmented LLM". It does not really matter. What does matter is that it's simple to implement, it does not require any frameworks, and it can solve for quite a few scenarios. It's plain old LLM function calling.

Here is one such implementation of the tool calling loop. It assumes the typical JSON format for the tool calls and uses the OpenAI chat completion API format.
I'm using the [`litellm`](https://github.com/BerriAI/litellm) library to call the OpenAI API since I can easily switch to another model (such as Anthropic)
and still use the same OpenAI API format. If you have never used `litellm` before that is fine! This is my first time using it. 
I only first heard about it when I  was reading about [smolagents](https://github.com/huggingface/smolagents) and how it utilizes it.
All you need to know is that `from litellm import completion` is the same as calling `chat.completions.create(...)` from the `openai` library.

In the loop below I also have some "print to console" functionality which uses [`rich`](https://rich.readthedocs.io/en/stable/introduction.html) under the hood.
I also borrowed this idea when looking through the source code of the [smolagents library](https://github.com/huggingface/smolagents) from Hugging Face. I will talk more about it later on in this post.


In [1]:
# | echo: false
# | warning: false
import dotenv

dotenv.load_dotenv()


from IPython.display import Markdown, display


def import_python_as_markdown(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return f"```python\n{content}\n```"


from tool_calling_loop import llm_with_tools, run_step

/Users/christopher/personal_projects/DrChrisLevy.github.io/posts/agents/env/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# | echo: false
file_path = "tool_calling_loop.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
import json
from concurrent import futures
from typing import Any, Callable, Dict

from litellm import completion
from utils import (
    console_print_llm_output,
    console_print_step,
    console_print_tool_call_inputs,
    console_print_tool_call_outputs,
    console_print_user_request,
)


def call_tool(tool: Callable, tool_args: Dict) -> Any:
    return tool(**tool_args)


def run_step(messages, tools=None, tools_lookup=None, model="gpt-4o-mini", **kwargs):
    messages = messages.copy()
    response = completion(model=model, messages=messages, tools=tools, **kwargs)
    response_message = response.choices[0].message.model_dump()
    response_message.pop("function_call", None)  # deprecated field in OpenAI API
    tool_calls = response_message.get("tool_calls", [])
    assistant_content = response_message.get("content", "")
    messages.append(response_message)

    if not tool_calls:
        response_message.pop("tool_calls", None)
        return messages

    tools_args_list = [json.loads(t["function"]["arguments"]) for t in tool_calls]
    tools_callables = [tools_lookup[t["function"]["name"]] for t in tool_calls]
    tasks = [(tools_callables[i], tools_args_list[i]) for i in range(len(tool_calls))]
    console_print_tool_call_inputs(assistant_content, tool_calls)
    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        tool_results = list(executor.map(lambda p: call_tool(p[0], p[1]), tasks))
    console_print_tool_call_outputs(tool_calls, tool_results)
    for tool_call, tool_result in zip(tool_calls, tool_results):
        messages.append(
            {
                "tool_call_id": tool_call["id"],
                "role": "tool",
                "content": str(tool_result),
                "name": tool_call["function"]["name"],
            }
        )
    return messages


def llm_with_tools(messages, tools=None, tools_lookup=None, model="gpt-4o-mini", max_steps=10, **kwargs):
    console_print_user_request(messages, model)
    done_calling_tools = False
    for counter in range(max_steps):
        console_print_step(counter)
        messages = run_step(messages, tools, tools_lookup, model=model, **kwargs)
        done_calling_tools = messages[-1]["role"] == "assistant" and messages[-1].get("content") and not messages[-1].get("tool_calls")
        if done_calling_tools:
            break
    console_print_llm_output(messages[-1]["content"])
    return messages

```

First we will run a single step, without any tools, which is a single LLM call.
Note that I return the entire message history in the output.

In [3]:
messages = [{"role": "user", "content": "Hello friend!"}]
run_step(messages)

[{'role': 'user', 'content': 'Hello friend!'},
 {'content': 'Hello! How can I assist you today?', 'role': 'assistant'}]

## Some Tools

Before going through an example task, let's show some initial tools. These tools are a list of functions that we can call.
We also have a lookup dictionary that maps the tool name to the tool function.


In [4]:
from tools import TOOL_LKP, TOOLS

TOOL_LKP

{'web_search': <function web_tools.web_search(query: str) -> str>,
 'execute_python_code': <function python_sandbox.execute_python_code(code: str, sandbox=None) -> dict>,
 'visit_web_page': <function web_tools.visit_web_page(url, max_chars: int = 20000, timeout: int = 10)>}

Let's see how each tool works first.

This first tool executes python code. It's actually running in a [Modal Sandbox](https://modal.com/docs/guide/sandbox) in a secure cloud container/environment. It's an awesome feature of Modal useful for executing arbitrary code. Let's skip the details for now and come back to it later. For now, just think of it as a way to execute python code and get back the results.

In [5]:
# This tool is a python code execution tool.
# The code is executed in a secure cloud container/environment using Modal.
# The results are returned locally as an object.
TOOL_LKP["execute_python_code"](code="print('Hello World!')")

{'stdout': 'Hello World!\n',
 'stderr': '',
 'success': True,
 'result': 'None',
 'error': None}

In [6]:
# We even get the last expression evaluated as a result just like in ipython repl
TOOL_LKP["execute_python_code"](code="import math; x = math.sqrt(4); print(x); y=2; x-y")

{'stdout': '2.0\nOut[1]: 0.0\n',
 'stderr': '',
 'success': True,
 'result': '0.0',
 'error': None}

The next tool uses `duckduckgo-search` to search the web.

In [7]:
TOOL_LKP["web_search"](query="What sporting events are happening today?")

[{'title': "Today's Top Sports Scores and Games (All Sports) | FOX Sports",
  'href': 'https://www.foxsports.com/scores',
  'body': "Visit FOXSports.com for today's top sports scores and games. Explore real-time game scores across MLB, NBA, NFL, Soccer, NHL and more."},
 {'title': 'Live Sports On TV Today - TV Guide',
  'href': 'https://www.tvguide.com/sports/live-today/',
  'body': "Here's sports to watch today, Thursday, Jan 23, 2025. ... coaches and celebrities are interviewed and discuss trending topics happening around the world. ... and he interviews various guests about ..."},
 {'title': 'Sports on TV today: Where to watch or stream games - Sports Media Watch',
  'href': 'https://www.sportsmediawatch.com/sports-on-tv-today-games-time-channel/',
  'body': 'See where to watch sports on TV today with this daily, updated guide of games and events on TV and streaming. This site may earn commission on subscriptions purchased via this page. For a full list of sports TV schedules, see t

And the next tool visits a web page and converts it to markdown.

In [8]:
print(TOOL_LKP["visit_web_page"](url="https://drchrislevy.github.io/"))

Chris Levy

[Chris Levy](./index.html)

* [About](./index.html)
* [Blog](./blog.html)

 
 

## On this page

* [About Me](#about-me)

# Chris Levy

 
[twitter](https://twitter.com/cleavey1985)
[Github](https://github.com/DrChrisLevy)
[linkedIn](https://www.linkedin.com/in/chris-levy-255210a4/)

**Hello!** I’m Chris Levy. I work in ML/AI and backend Python development.

## About Me

I spent a good amount of time in school where I completed a PhD in applied math back in 2015. After graduating I shifted away from academia and started working in industry. I mostly do backend python development these days, and build ML/AI applications/services. I work across the entire stack from research, to training and evaluating models, to deploying models, and getting in the weeds of the infrastructure and devops pipelines.

Outside of AI/ML stuff, I enjoy spending time with my family and three kids, working out, swimming, cycling, and playing guitar.

![](pic_me.jpeg)


To pass these tools to the LLM, we use the typical JSON format used within the OpenAI API format.

In [9]:
TOOLS

[{'type': 'function',
  'function': {'name': 'execute_python_code',
   'description': 'Run and execute the python code and return the results.',
   'parameters': {'type': 'object',
    'properties': {'code': {'type': 'string',
      'description': 'The python code to execute.'}},
    'required': ['code']}}},
 {'type': 'function',
  'function': {'name': 'web_search',
   'description': 'Search the web for the query and return the results.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'The query to search for.'}},
    'required': ['query']}}},
 {'type': 'function',
  'function': {'name': 'visit_web_page',
   'description': 'Visit the web page and return the results.',
   'parameters': {'type': 'object',
    'properties': {'url': {'type': 'string',
      'description': 'The URL to visit.'}},
    'required': ['url']}}}]

## Example Task 1

Okay, so let's run the tool calling loop now with the tools defined above to illustrate how it works.
Here is a task where we ask some questions about recent NBA events.



In [10]:
task = """
        Recently on Jan 2 2025, Steph Curry made a series of 3 pointers in one game without missing. 
        How many three pointers did he make in total that game?
        How many points did he score in total that game?
        How many combined points did both teams score on that game?
        Of the total points scored by both teams, what percentage was made by Steph Curry?

        One more task. Lebron James also played a game on Jan 2 2025.
        How old is Lebron James and how many points did he score in his game on Jan 2 2025?
        Take his total points scored that game and raise it to the power of 5. What is the result?
    
        """
messages = [
    {
        "role": "system",
        "content": """You are a helpful assistant. Use the supplied tools to assist the user. 
        Always use python to do math. After getting web search results be sure to visit the web page and convert it to markdown. 
        Todays date is 2025-01-03. Remember to give a final answer in your last message answering all of the user's questions.""",
    },
    {
        "role": "user",
        "content": task,
    },
]

There is an answer to this question. Here is the correct ground truth answer.

In [11]:
example_one_answer = """
Game stats from January 2, 2025:

Steph Curry:
- Made 8 three pointers
- Total points: 30
- Game final score: Warriors 139, 76ers 105 so the total points scored by both teams is 244
- Curry's percentage of total points: 30/244 ~= 12.3%

Lebron James on January 2, 2025:
- Age: 40
- Points scored: 38
- Points scored raised to the power of 5: 38^5 = 79,235,168
"""

Let's also have a simple LLM call to evaluate if a response is correct.



In [12]:
import json


def eval_example_one(input_answer):
    input_msgs = [
        {
            "role": "user",
            "content": f"""
         
Original question:
{messages[-1]["content"]}

Here is the ground truth answer:
{example_one_answer}

Here is the predicted answer from an LLM.
{input_answer}

Given the context of the correct answer and question, did the LLM get everything correct in its predicted answer?
Return True or False. Only return True if the LLM got everything correct
and answered each part of the question correctly. Also give an explanation of why you returned True or False.
Output JSON.

{{
    "correct": True or False,
    "explanation": "explanation of why you returned True or False"
}}
""",
        },
    ]

    return json.loads(run_step(input_msgs, model="gpt-4o", response_format={"type": "json_object"})[-1]["content"])


# Example of incorrect answer
print(eval_example_one("Lebron James is 40 years old and scored 38 points in his game on Jan 2 2025."))

# Example of correct answer
print(
    eval_example_one(
        "Lebron James is 40 years old and scored 38 points in his game on Jan 2 2025. 38 to the power of 5 is 79,235,168.  Steph scored 30, made 8 three pointers without missing. The total points scored by both teams was 244 and Steph scored 12.3 percent of the total points."
    )
)

{'correct': False, 'explanation': "The LLM correctly identified LeBron James's age as 40 and his points scored as 38 on January 2, 2025. However, the LLM did not address or verify the other components of the original question, specifically regarding Steph Curry's performance and game statistics, nor did it calculate the result of raising LeBron's points to the power of 5. Therefore, not all aspects of the original question were answered, and the LLM's response is incomplete, leading to a determination of False."}
{'correct': True, 'explanation': "The LLM provided the same answers as the ground truth for each part of the question. Steph Curry made 8 three pointers without missing and scored a total of 30 points. The combined score for both teams was 244, and Steph Curry's points accounted for approximately 12.3% of the total. LeBron James was 40 years old on January 2, 2025, and scored 38 points in his game on that day. When 38 is raised to the power of 5, the result is 79,235,168. Ther

### gpt-4o-mini

Okay, lets send this same task to `gpt-4o-mini` and see how it does.

In [13]:
messages_final = llm_with_tools(messages, model="gpt-4o-mini", tools=TOOLS, tools_lookup=TOOL_LKP)

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│         Recently on Jan 2 2025, Steph Curry made a series of 3 pointers in one game without missing.            │
│         How many three pointers did he make in total that game?                                                 │
│         How many points did he score in total that game?                                                        │
│         How many combined points did both teams score on that game?                                             │
│         Of the total points scored by both teams, what percentage was made by Steph Curry?                      │
│                                                                                                                 │
│         One more task. Lebron James also played a game on Jan 2 2025.                                           │
│         How old is Lebron James and how many points did he score in his game on Jan 2 2025?                     │
│         Take his total points scored that game and raise it to the power of 5. What is the result?              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─ gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Assistant Content ─────────────────────────────────────────────╮ │
│ │ Assistant Message:                                                                                          │ │
│ │  None                                                                                                       │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │ {'query': 'Steph Curry Jan 2 2025 three pointers total points scored'}                                      │ │
│ ╰─ web_search - call_tjg5CMKv8PxBmin6lFW0WVQt ────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │ {'query': 'LeBron James age and points scored on Jan 2 2025'}                                               │ │
│ ╰─ web_search - call_jz00I0lIXewPjmYlrkIPVCNU ────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ Result 1:                                                                                                   │ │
│ │ Title: Steph Curry perfect from 3 as Warriors blow out 76ers - The Mercury News                             │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.mercurynews.com/2025/01/02/curry-perfect-from-3-as-warriors-start-2025-hot-in-blowout-win-over- │ │
│ │ 76ers/                                                                                                      │ │
│ │ Steph Curry went 8-for-98 from 3-point land as the Warriors routed Philadelphia. ... Curry went 8-for-8     │ │
│ │ from 3-point land and scored 30 points. Share ...                                                           │ │
│ │                                                                                                             │ │
│ │ Result 2:                                                                                                   │ │
│ │ Title: Stephen Curry scores 21 points in Warriors' 131-106 victory over the ...                             │ │
│ │ URL: https://abc7news.com/post/stephen-curry-scores-21-points-warriors-131-106-victory-bulls/15831913/      │ │
│ │ Friday, January 24, 2025 5:36AM automation SAN FRANCISCO -- - Stephen Curry had 21 points and seven         │ │
│ │ assists, Quinten Post scored a career-high 20 poin...                                                       │ │
│ │                                                                                                             │ │
│ │ Result 3:                                                                                                   │ │
│ │ Title: Warriors blow out Sixers behind Curry's perfect three-point night                                    │ │
│ │ URL: https://www.goldenstateofmind.com/2025/1/2/24334774/currys-perfect-three-point-night-overwhelms-76ers  │ │
│ │ By Sean Keane @seankeane Jan 2, 2025, 9:47pm ... Steph Curry shot 8-for-8 on three-pointers on his way to   │ │
│ │ 30 points and 10 assists and seven Warriors ...                                                             │ │
│ │                                                                                                             │ │
│ │ Result 4:                                                                                                   │ │
│ │ Title: Steph Curry Stats In January 2025 - StatMuse                                                         │ │
│ │ URL: https://www.statmuse.com/nba/ask/steph-curry-stats-in-january-2025                                     │ │
│ │ Stephen Curry has averaged 29.0 points, 6.7 rebounds and 3.3 assists in 3 games this month. ... Stephen     │ │
│ │ Curry has averaged 29.0 points, 6.7 rebounds a...                                                           │ │
│ │                                                                                                             │ │
│ │ Result 5:                                                                                                   │ │
│ │ Title: Stephen Curry three point record - Stephen Curry makes NBA history with ...                          │ │
│ │ URL:                                                                                                        │ │
│ │ https://thesportstak.com/basketball/story/stephen-curry-makes-nba-history-with-career-milestones-against-ch │ │
│ │ icago-bulls-3157385-2025-01-24                                                                              │ │
│ │ In the first half of the game against the Bulls, Curry knocked down a three-pointer that pushed his career  │ │
│ │ total to over 3,900 made three-pointers. In...      

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Assistant Content ─────────────────────────────────────────────╮ │
│ │ Assistant Message:                                                                                          │ │
│ │  None                                                                                                       │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │   1 total_points_both_teams = 139 + 105                                                                     │ │
│ │   2 total_points_curry = 30                                                                                 │ │
│ │   3 percentage_curry = (total_points_curry / total_points_both_teams) * 100                                 │ │
│ │   4 lebron_points = 21                                                                                      │ │
│ │   5 lebron_age = 40  # LeBron James's age                                                                   │ │
│ │   6 lebron_points_power_5 = lebron_points ** 5                                                              │ │
│ │   7 total_points_both_teams, percentage_curry, lebron_age, lebron_points_power_5                            │ │
│ ╰─ execute_python_code - call_RYyPWSmdTvMq5sntI88ISLae ───────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ stdout:                                                                                                     │ │
│ │ Out[1]: (244, 12.295081967213115, 40, 4084101)                                                              │ │
│ │                                                                                                             │ │
│ │ is_success: True                                                                                            │ │
│ ╰─ execute_python_code - call_RYyPWSmdTvMq5sntI88ISLae ───────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Here's the information about the games played by Steph Curry and LeBron James on January 2, 2025:               │
│                                                                                                                 │
│                                                   Steph Curry                                                   │
│                                                                                                                 │
│  • Total Three-Pointers Made: 8 (Curry went 8-for-8 from three-point range)                                     │
│  • Total Points Scored: 30 points                                                                               │
│  • Combined Points Scored by Both Teams: 244 points (Warriors: 139, 76ers: 105)                                 │
│  • Percentage of Points Scored by Steph Curry: Approximately 12.30%                                             │
│                                                                                                                 │
│                                                  LeBron James                                                   │
│                                                                                                                 │
│  • Age: 40 years old                                                                                            │
│  • Points Scored in His Game: 21 points                                                                         │
│  • Result of Raising His Points to the Power of 5: 4,084,101                                                    │
│                                                                                                                 │
│ This summarizes the performances of both players on that date. Let me know if you need any more information!    │
╰─ final-answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

We can look at all the messages in the final output, which includes all the messages handled by the LLM.



In [14]:
# Commenting out since the output is long from the webpages visited.
# But has all the messages chat history and tool calls in the OpenAI API format.

# messages_final

Let's use our LLM judge to evaluate the final output.

In [15]:
eval_example_one(messages_final[-1]["content"])

{'correct': False,
 'explanation': "The LLM did not get everything correct. While it correctly noted Steph Curry's total three-pointers, total points scored, combined points scored by both teams, and the percentage of total points scored by Curry, it made an error in LeBron James's performance. The LLM stated that LeBron James scored 21 points in his game, but the ground truth indicates he scored 38 points. Consequently, raising 21 (the incorrect point total) to the power of 5 yields an incorrect result of 4,084,101, whereas the correct computation for 38 points raised to the power of 5 should be 79,235,168. Additionally, the age of LeBron James was correctly noted as 40 years old. The miscalculation for LeBron James's points means the LLM did not answer each part of the question correctly."}

### claude-3-5-sonnet {#sec-claude-3-5-sonnet-ex1}

Let's send this same task to Anthropic's `claude-3-5-sonnet` model.
That's the beauty of `litellm`! We can easily switch between models and still use the same all familiar OpenAI API format.

In [16]:
messages_final = llm_with_tools(messages, model="claude-3-5-sonnet-20240620", tools=TOOLS, tools_lookup=TOOL_LKP)

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│         Recently on Jan 2 2025, Steph Curry made a series of 3 pointers in one game without missing.            │
│         How many three pointers did he make in total that game?                                                 │
│         How many points did he score in total that game?                                                        │
│         How many combined points did both teams score on that game?                                             │
│         Of the total points scored by both teams, what percentage was made by Steph Curry?                      │
│                                                                                                                 │
│         One more task. Lebron James also played a game on Jan 2 2025.                                           │
│         How old is Lebron James and how many points did he score in his game on Jan 2 2025?                     │
│         Take his total points scored that game and raise it to the power of 5. What is the result?              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─ claude-3-5-sonnet-20240620 ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Assistant Content ─────────────────────────────────────────────╮ │
│ │ Assistant Message:                                                                                          │ │
│ │  To answer these questions, I'll need to search for information about Steph Curry's and LeBron James'       │ │
│ │ recent games, as well as LeBron's age. Let's start by searching for Steph Curry's game on January 2, 2025.  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │ {'query': 'Steph Curry 3 pointers game January 2, 2025'}                                                    │ │
│ ╰─ web_search - toolu_01HsnLFmSZwKzigDLMvBzWwP ───────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ Result 1:                                                                                                   │ │
│ │ Title: Steph Curry perfect from 3 as Warriors blow out 76ers - The Mercury News                             │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.mercurynews.com/2025/01/02/curry-perfect-from-3-as-warriors-start-2025-hot-in-blowout-win-over- │ │
│ │ 76ers/                                                                                                      │ │
│ │ Steph Curry went 8-for-98 from 3-point land as the Warriors routed Philadelphia. ... Jan. 2, 2025. (Ray     │ │
│ │ Chavez/Bay Area News Group) ... who passed Mic...                                                           │ │
│ │                                                                                                             │ │
│ │ Result 2:                                                                                                   │ │
│ │ Title: Warriors 139-105 76ers (Jan 2, 2025) Final Score - ESPN                                              │ │
│ │ URL: https://www.espn.com/nba/game/_/gameId/401705033/76ers-warriors                                        │ │
│ │ — Stephen Curry had 30 points and made all eight of his 3-point attempts, and the Golden State Warriors     │ │
│ │ beat the Philadelphia 76ers 139-105 on Thursda...                                                           │ │
│ │                                                                                                             │ │
│ │ Result 3:                                                                                                   │ │
│ │ Title: Steph Curry hits 'insane' milestone hours after NBA All-Star Game nod ...                            │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.nbcsportsbayarea.com/nba/golden-state-warriors/steph-curry-3pointers-milestone-chicago-bulls/18 │ │
│ │ 23085/                                                                                                      │ │
│ │ — Warriors on NBCS (@NBCSWarriors) January 24, 2025. For a while, it looked like Curry's efforts might be   │ │
│ │ for naught. One night after blowing a 17-poi...                                                             │ │
│ │                                                                                                             │ │
│ │ Result 4:                                                                                                   │ │
│ │ Title: Ranking each of Steph Curry's 3s in a perfect 8-for-8 game                                           │ │
│ │ URL: https://www.mercurynews.com/2025/01/03/ranking-each-of-steph-currys-3s-in-a-perfect-8-for-8-game/      │ │
│ │ Golden State Warriors' Stephen Curry (30) celebrates after making a three-point basket against the          │ │
│ │ Philadelphia 76ers in the third quarter of an NBA g...                                                      │ │
│ │                                                                                                             │ │
│ │ Result 5:                                                                                                   │ │
│ │ Title: Stephen Curry 2025 Game Log - StatMuse                                                               │ │
│ │ URL: https://www.statmuse.com/nba/player/stephen-curry-787/game-log?seasonYear=2025                         │ │
│ │ Stephen Curry most points in a game ; Stephen Curry stats in the 2015 playoffs ; Stephen Curry most steals  │ │
│ │ in a game ; See trending More Warriors Stat...      

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Assistant Content ─────────────────────────────────────────────╮ │
│ │ Assistant Message:                                                                                          │ │
│ │  Based on the search results, we have information about LeBron James' game on January 2, 2025. Let's answer │ │
│ │ your questions:                                                                                             │ │
│ │                                                                                                             │ │
│ │ 5. How old is LeBron James and how many points did he score in his game on Jan 2 2025?                      │ │
│ │                                                                                                             │ │
│ │ LeBron James' age: As of January 2, 2025, LeBron James would be 40 years old (born on December 30, 1984).   │ │
│ │                                                                                                             │ │
│ │ Points scored: According to the game recap, LeBron James scored 38 points in the game against the Portland  │ │
│ │ Trail Blazers on January 2, 2025.                                                                           │ │
│ │                                                                                                             │ │
│ │ 6. Take his total points scored that game and raise it to the power of 5. What is the result?               │ │
│ │                                                                                                             │ │
│ │ Let's calculate this using Python:                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │   1 lebron_points = 38                                                                                      │ │
│ │   2 result = lebron_points ** 5                                                                             │ │
│ │   3 print(f"LeBron's points ({lebron_points}) raised to the power of 5 is: {result:,}")                     │ │
│ ╰─ execute_python_code - toolu_017PLS5Bc5d3SYcwe7U4rNGY ──────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ stdout:                                                                                                     │ │
│ │ LeBron's points (38) raised to the power of 5 is: 79,235,168                                                │ │
│ │                                                                                                             │ │
│ │ is_success: True                                                                                            │ │
│ ╰─ execute_python_code - toolu_017PLS5Bc5d3SYcwe7U4rNGY ──────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ LeBron James' points (38) raised to the power of 5 is 79,235,168.                                               │
│                                                                                                                 │
│ To summarize the answers to all your questions:                                                                 │
│                                                                                                                 │
│  1 Steph Curry made 8 three-pointers in the game on January 2, 2025.                                            │
│  2 Steph Curry scored a total of 30 points in that game.                                                        │
│  3 The combined points scored by both teams (Warriors and 76ers) was 244 points.                                │
│  4 Steph Curry scored 12.30% of the total points in the game.                                                   │
│  5 LeBron James was 40 years old on January 2, 2025, and he scored 38 points in his game against the Portland   │
│    Trail Blazers on that date.                                                                                  │
│  6 LeBron's points (38) raised to the power of 5 is 79,235,168.                                                 │
╰─ final-answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [17]:
eval_example_one(messages_final[-1]["content"])

{'correct': True,
 'explanation': "The LLM correctly provided the number of three-pointers made by Steph Curry, his total points, the combined score of both teams, and the percentage of total points he scored. It also accurately stated LeBron James' age, points scored in his game, and the calculation of his points raised to the power of 5. Therefore, the LLM answered each part of the question correctly."}

### deepseek/deepseek-chat

We can also try the same task with `"deepseek/deepseek-chat"`.

In [18]:
messages_final = llm_with_tools(messages, model="deepseek/deepseek-chat", tools=TOOLS, tools_lookup=TOOL_LKP)

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│         Recently on Jan 2 2025, Steph Curry made a series of 3 pointers in one game without missing.            │
│         How many three pointers did he make in total that game?                                                 │
│         How many points did he score in total that game?                                                        │
│         How many combined points did both teams score on that game?                                             │
│         Of the total points scored by both teams, what percentage was made by Steph Curry?                      │
│                                                                                                                 │
│         One more task. Lebron James also played a game on Jan 2 2025.                                           │
│         How old is Lebron James and how many points did he score in his game on Jan 2 2025?                     │
│         Take his total points scored that game and raise it to the power of 5. What is the result?              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─ deepseek/deepseek-chat ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Assistant Content ─────────────────────────────────────────────╮ │
│ │ Assistant Message:                                                                                          │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │ {'query': 'Steph Curry 3 pointers Jan 2 2025 game'}                                                         │ │
│ ╰─ web_search - call_0_db2deee7-8ebb-4816-85f1-d57eac016a71 ──────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │ {'query': 'Lebron James age Jan 2 2025 and points scored in game'}                                          │ │
│ ╰─ web_search - call_1_3ecd9dd9-bb99-49ad-aee1-828c03993f4f ──────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ Result 1:                                                                                                   │ │
│ │ Title: Stephen Curry DOESN'T MISS From Three vs. 76ers | Jan. 2, 2025                                       │ │
│ │ URL: https://www.youtube.com/watch?v=eZW5n62DbjE                                                            │ │
│ │ Stephen Curry went 8 of 8 from three-point range as the Golden State Warriors defeated the Philadelphia     │ │
│ │ 76ers 139-105.                                                                                              │ │
│ │                                                                                                             │ │
│ │ Result 2:                                                                                                   │ │
│ │ Title: Steph Curry perfect from 3 as Warriors blow out 76ers - The Mercury News                             │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.mercurynews.com/2025/01/02/curry-perfect-from-3-as-warriors-start-2025-hot-in-blowout-win-over- │ │
│ │ 76ers/                                                                                                      │ │
│ │ Despite sprained thumb, Curry perfect from 3 as Warriors start 2025 hot in blowout win over 76ers Playing   │ │
│ │ with his thumb wrapped, Curry went 8-for-8 f...                                                             │ │
│ │                                                                                                             │ │
│ │ Result 3:                                                                                                   │ │
│ │ Title: Steph Curry Made NBA History In Bulls-Warriors Game                                                  │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.si.com/fannation/nba/fastbreak/steph-curry-made-nba-history-in-bulls-warriors-game-thursday-new │ │
│ │ s                                                                                                           │ │
│ │ Steph Curry had 11 points, one rebound and five assists while shooting 4/9 from the field and 3/6 from the  │ │
│ │ three-point range in his first 16 minutes o...                                                              │ │
│ │                                                                                                             │ │
│ │ Result 4:                                                                                                   │ │
│ │ Title: Warriors 139-105 76ers (Jan 2, 2025) Final Score - ESPN                                              │ │
│ │ URL: https://www.espn.com/nba/game/_/gameId/401705033                                                       │ │
│ │ — Stephen Curry had 30 points and made all eight of his 3-point attempts, and the Golden State Warriors     │ │
│ │ beat the Philadelphia 76ers 139-105 on Thursda...                                                           │ │
│ │                                                                                                             │ │
│ │ Result 5:                                                                                                   │ │
│ │ Title: NBA: Stephen Curry tastes 3-point perfection in Warriors win                                         │ │
│ │ URL: https://sports.inquirer.net/600979/nba-stephen-curry-tastes-3-point-perfection-in-warriors-win         │ │
│ │ Golden State Warriors guard Stephen Curry (30) celebrates with forward Draymond Green after making a        │ │
│ │ 3-point basket during the second half of an NBA b...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Assistant Content ─────────────────────────────────────────────╮ │
│ │ Assistant Message:                                                                                          │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │    1 # Steph Curry's game                                                                                   │ │
│ │    2 three_pointers_made = 8                                                                                │ │
│ │    3 total_points = 30                                                                                      │ │
│ │    4 combined_points = 139 + 105                                                                            │ │
│ │    5 percentage_points = (total_points / combined_points) * 100                                             │ │
│ │    6                                                                                                        │ │
│ │    7 # LeBron James' game                                                                                   │ │
│ │    8 lebron_age = 40                                                                                        │ │
│ │    9 lebron_points = 38                                                                                     │ │
│ │   10 lebron_points_power_5 = lebron_points ** 5                                                             │ │
│ │   11                                                                                                        │ │
│ │   12 result = {                                                                                             │ │
│ │   13     'steph_curry': {                                                                                   │ │
│ │   14         'three_pointers_made': three_pointers_made,                                                    │ │
│ │   15         'total_points': total_points,                                                                  │ │
│ │   16         'combined_points': combined_points,                                                            │ │
│ │   17         'percentage_points': percentage_points                                                         │ │
│ │   18     },                                                                                                 │ │
│ │   19     'lebron_james': {                                                                                  │ │
│ │   20         'age': lebron_age,                                                                             │ │
│ │   21         'points_scored': lebron_points,                                                                │ │
│ │   22         'points_power_5': lebron_points_power_5                                                        │ │
│ │   23     }                                                                                                  │ │
│ │   24 }                                                                                                      │ │
│ │   25 result                                                                                                 │ │
│ ╰─ execute_python_code - call_0_7a5e8640-3e4a-4490-b05a-96b3d0a024fe ─────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ stdout:                                                                                                     │ │
│ │ Out[1]:                                                                                                     │ │
│ │ {'steph_curry': {'three_pointers_made': 8,                                                                  │ │
│ │   'total_points': 30,                                                                                       │ │
│ │   'combined_points': 244,                                                                                   │ │
│ │   'percentage_points': 12.295081967213115},                                                                 │ │
│ │  'lebron_james': {'age': 40, 'points_scored': 38, 'points_power_5': 79235168}}                              │ │
│ │                                                                                                             │ │
│ │ is_success: True                                                                                            │ │
│ ╰─ execute_python_code - call_0_7a5e8640-3e4a-4490-b05a-96b3d0a024fe ─────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Here are the answers to your questions:                                                                         │
│                                                                                                                 │
│                                       Steph Curry's Game on Jan 2, 2025:                                        │
│                                                                                                                 │
│  • Three-pointers made: 8                                                                                       │
│  • Total points scored: 30                                                                                      │
│  • Combined points scored by both teams: 244                                                                    │
│  • Percentage of total points scored by Steph Curry: 12.30%                                                     │
│                                                                                                                 │
│                                       LeBron James' Game on Jan 2, 2025:                                        │
│                                                                                                                 │
│  • Age: 40                                                                                                      │
│  • Points scored: 38                                                                                            │
│  • Points scored raised to the power of 5: 79,235,168                                                           │
│                                                                                                                 │
│ Let me know if you need further assistance!                                                                     │
╰─ final-answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
eval_example_one(messages_final[-1]["content"])

{'correct': True,
 'explanation': 'The LLM correctly answered all parts of the original question. It provided the number of three-pointers made by Steph Curry, his total points, the combined points scored by both teams, and the percentage of total points scored by Curry. Additionally, for LeBron James, it correctly stated his age, the points he scored, and the result of raising his points to the power of 5. Therefore, the predicted answer matches the ground truth for all aspects of the question.'}

# ReAct

One of the main prompting techniques for building agents comes from the paper --> [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/pdf/2210.03629).
It is also the approach `smolagents` uses in their library as talked about in their conceptual guide [here](https://github.com/huggingface/smolagents/blob/main/docs/source/en/conceptual_guides/react.md).
I'm sure a lot of other frameworks use this approach, or modified versions of it, as well. You should check out the `smolagents` library, documentation, and code for more [details](https://huggingface.co/blog/smolagents). 

The ReAct prompting framework (short for Reasoning and Acting) is a technique designed to enhance the capabilities of large language model (LLM) agents by enabling them to reason and act iteratively when solving complex tasks. ReAct combines chain-of-thought reasoning with decision making actions, allowing the model to think step by step while simultaneously interacting with the environment to gather necessary information.

The key elements of ReAct are:

**Reasoning**: The model generates intermediate steps to explain its thought process while solving a problem or addressing a task.

**Acting**: The model performs actions based on its reasoning i.e. calling tools. 

**Observation**: The outputs of actions (tool calls) provide feedback or data to guide the next reasoning step.

**Iterative Process**: ReAct operates in a loop, where the outputs of reasoning and acting are used to refine the approach, gather additional information, or confirm conclusions until the task is resolved.

It's some what similar to what we saw above in the *Tool calling Loop* @sec-tool_calling_loop. Actually, when you compare the output from our first example task in the tool calling loop, you can see that `"anthropic/claude-3-5-sonnet"` @sec-claude-3-5-sonnet-ex1 is quite verbose in explaining its reasoning while making tool calls. It's already using some sort of chain of thought reasoning. However the OpenAI `gpt-4o-mini` model does not output much in the way of reasoning.

Let's see if we can implement a simple version of ReAct prompting. The goal here is not to be robust as a framework, but rather to illustrate some of the concepts for educational purposes.
I have a system prompt explaining ReAct with some examples, followed by code to run a step and run a loop. It's similar in structure to the tool calling loop.
I have simplified things here by assuming only one tool call is made in each step. I have also chosen to use structured JSON output for all the assistant messages
using the OpenAI API format. I am using the same tools as before but I have added a `final_answer` tool call.

In [20]:
# | echo: false
file_path = "react.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
import json
from typing import Any, Callable, Dict

from litellm import completion
from tools import TOOL_LKP
from utils import console_print_react_tool_action_inputs, console_print_react_tool_action_outputs, console_print_user_request

REACT_SYSTEM_PROMPT = """
You are a helpful assistant that uses reasoning and actions to solve tasks step by step. 
You have access to the following tools:

[{'type': 'function',
  'function': {'name': 'execute_python_code',
   'description': 'Run and execute the python code and return the results.',
   'parameters': {'type': 'object',
    'properties': {'code': {'type': 'string',
      'description': 'The python code to execute.'}},
    'required': ['code']}}},
 {'type': 'function',
  'function': {'name': 'web_search',
   'description': 'Search the web for the query and return the results.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'The query to search for.'}},
    'required': ['query']}}},
 {'type': 'function',
  'function': {'name': 'visit_web_page',
   'description': 'Visit the web page and return the results.',
   'parameters': {'type': 'object',
    'properties': {'url': {'type': 'string',
      'description': 'The URL to visit.'}},
    'required': ['url']}}},
 {'type': 'function',
  'function': {'name': 'final_answer',
   'description': 'Return the final answer to the task.',
   'parameters': {'type': 'object',
    'properties': {'answer': {'type': 'string',
      'description': 'The final answer to the task.'}},
    'required': ['answer']}}}
]

For each step, you should:

1. Think: Explain your thought process and what you plan to do next
2. Act: Call one of the available tools using the proper JSON format. Only call one tool at a time.
3. Observe: Review the results from the tool call
4. Repeat or Conclude: Either take another step or provide your final answer

YOU MUST ALWAYS RESPOND IN STRUCTURED JSON FORMAT.
The output format must follow this Schema:

{
  "THOUGHT": "Your thought process",
  "ACTION": {
    "tool_name": "The name of the tool to call",
    "tool_arguments": "The arguments to pass to the tool"
  }
}

Here are some examples of how to solve tasks:
Example 1: "What was the average temperature in New York City last week?"

{
  "THOUGHT": "I need to search for NYC weather data from the past week.",
  "ACTION": {
    "tool_name": "web_search",
    "tool_arguments": {
      "query": "NYC weather data December 27-January 2 2025"
    }
  }
}

< wait for tool call to finish >

OBSERVATION: [
    {
        "title": "Historical Weather Data - NYC Central Park",
        "href": "https://weather.example.com/nyc/history",
        "body": "View detailed temperature records for New York City Central Park weather station..."
    }
]

{
  "THOUGHT": "Let me visit the webpage to get the detailed temperature data.",
  "ACTION": {
    "tool_name": "visit_web_page",
    "tool_arguments": {
      "url": "https://weather.example.com/nyc/history"
    }
  }
}

< wait for tool call to finish >

OBSERVATION: # NYC Central Park Weather History
## December 27, 2024 - January 2, 2025

Daily Temperature Records (°F):
- December 27: Average 35°
- December 28: Average 31.5°
- December 29: Average 37.5°
- December 30: Average 35°
- December 31: Average 29°
- January 1: Average 32.5°
- January 2: Average 37°

{
   "THOUGHT": "Now I can calculate the overall average temperature using Python.",
   "ACTION": {
       "tool_name": "execute_python_code",
       "tool_arguments": {
           "code": "temps = [35, 31.5, 37.5, 35, 29, 32.5, 37]\navg_temp = sum(temps) / len(temps)\nprint(f\"Average temperature: {avg_temp:.1f}°F\")"
       }
   }
}

< wait for tool call to finish >

OBSERVATION:
Average temperature: 33.9°F

{
   "THOUGHT": "I have the average temperature and can provide the final answer.",
   "ACTION": {
       "tool_name": "final_answer",
       "tool_arguments": {
           "answer": "Based on verified data from the NYC Central Park weather station, the average temperature in NYC last week was 33.9°F."
       }
   }
}

Example 2: "Find the latest news about AI regulation in the EU and summarize the key points"

{
   "THOUGHT": "I should search for recent news about EU AI regulation.",
   "ACTION": {
       "tool_name": "web_search",
       "tool_arguments": {
           "query": "latest EU AI regulation news 2025"
       }
   }
}

< wait for tool call to finish >

OBSERVATION: [
    {
        "title": "EU finalizes historic AI regulation law",
        "href": "https://example.com/eu-ai-law",
        "body": "The European Union has reached a landmark agreement on comprehensive AI regulations..."
    },
    {
        "title": "What businesses need to know about EU AI Act",
        "href": "https://example.com/eu-ai-business-impact",
        "body": "The new legislation will require companies to..."
    }
]

{
   "THOUGHT": "The first result seems most relevant. I should visit that webpage to get the full details.",
   "ACTION": {
       "tool_name": "visit_web_page",
       "tool_arguments": {
           "url": "https://example.com/eu-ai-law"
       }
   }
}

< wait for tool call to finish >

OBSERVATION: # EU Finalizes Historic AI Regulation Law

The European Union has reached a landmark agreement on comprehensive AI regulations, marking the world's first major legislation on artificial intelligence. The law, known as the AI Act, introduces strict rules for AI systems based on their level of risk.

## Key Points:
1. Risk-based approach to AI regulation
2. Strict rules for high-risk AI applications
3. Ban on certain AI uses like social scoring
...

{
   "THOUGHT": "Now I have detailed information from the webpage and can provide a comprehensive summary.",
   "ACTION": {
       "tool_name": "final_answer",
       "tool_arguments": {
           "answer": "According to recent news, the EU has finalized its AI Act with these key points: [detailed summary from webpage content]..."
       }
   }
}

Example 3: "Calculate the compound interest on $1000 invested for 5 years at 7% annual interest rate"

{
   "THOUGHT": "I need to use Python to calculate compound interest using the formula A = P(1 + r)^t",
   "ACTION": {
       "tool_name": "execute_python_code",
       "tool_arguments": {
           "code": "principal = 1000\nrate = 0.07\ntime = 5\namount = principal * (1 + rate) ** time\ninterest = amount - principal\nprint(f'Total amount: ${amount:.2f}')\nprint(f'Interest earned: ${interest:.2f}')"
       }
   }
}

< wait for tool call to finish >

OBSERVATION:
Total amount: $1402.55
Interest earned: $402.55

{
   "THOUGHT": "I have the calculation results and can provide the final answer.",
   "ACTION": {
       "tool_name": "final_answer",
       "tool_arguments": {
           "answer": "A $1000 investment at 7% annual interest for 5 years would grow to $1402.55, earning $402.55 in compound interest."
       }
   }
}

Example 4: "Find today's most popular movie on Rotten Tomatoes and its audience score"

{
   "THOUGHT": "Let me try to scrape the Rotten Tomatoes homepage using BeautifulSoup.",
   "ACTION": {
       "tool_name": "execute_python_code",
       "tool_arguments": {
           "code": "from bs4 import BeautifulSoup\nimport requests\n\nurl = 'https://www.rottentomatoes.com'\nresponse = requests.get(url)\nsoup = BeautifulSoup(response.text, 'html.parser')"
       }
   }
}

< wait for tool call to finish >

OBSERVATION:
{
    'stdout': '',
    'stderr': "ModuleNotFoundError: No module named 'bs4'",
    'success': False,
    'error': "ModuleNotFoundError: No module named 'bs4'"
}

{
   "THOUGHT": "Ah, we need to install beautifulsoup4 first. Let's install it and then scrape the data.",
   "ACTION": {
       "tool_name": "execute_python_code",
       "tool_arguments": {
           "code": "!pip install beautifulsoup4\nfrom bs4 import BeautifulSoup\nimport requests\n\nurl = 'https://www.rottentomatoes.com'\nresponse = requests.get(url)\nsoup = BeautifulSoup(response.text, 'html.parser')\n\n# Find the trending movie section\ntrending = soup.find('div', class_='trending-movie')\nmovie = trending.find('h2').text.strip()\nscore = trending.find('span', class_='audience-score').text.strip()\n\nprint(f\"Most Popular Movie: {movie}\")\nprint(f\"Audience Score: {score}\")"
       }
   }
}

< wait for tool call to finish >

OBSERVATION:
Collecting beautifulsoup4
  ...
Successfully installed beautifulsoup4-4.12.2
Most Popular Movie: Dune: Part Two
Audience Score: 97%

{
   "THOUGHT": "Great! We successfully installed BeautifulSoup and got the movie data.",
   "ACTION": {
       "tool_name": "final_answer",
       "tool_arguments": {
           "answer": "The most popular movie on Rotten Tomatoes today is 'Dune: Part Two' with an audience score of 97%. After encountering and fixing a missing package error, we were able to successfully scrape this data from the Rotten Tomatoes homepage."
       }
   }
}


Important rules:
1. Always explain your reasoning in the THOUGHT step
2. Use proper JSON format for tool calls in the ACTION step and only call one tool at a time.
3. Only use the available tools (web_search, visit_web_page, execute_python_code, final_answer)
4. Make your final answer using the "final_answer" tool to signal the end of the task
5. Break down complex tasks into smaller steps
6. Use Python code execution for any calculations
7. If a tool call fails, explain why in your next thought and try a different approach
8. Don't make assumptions - verify information when needed
9. Always review tool outputs before proceeding to next steps
10. When searching the web, follow up relevant results with visit_web_page to get detailed information
11. Remember that web_search returns a list of results with titles, URLs, and snippets
12. Remember that visit_web_page returns markdown-formatted content
13. If you encounter an error (website blocked, code syntax error, etc.), explain the error and try an alternative approach
14. Keep track of failed attempts and avoid repeating the same unsuccessful approach

Remember: Today's date is 2025-01-03."""


def final_answer(answer):
    return answer


TOOL_LKP["final_answer"] = final_answer


def call_tool(tool: Callable, tool_args: Dict) -> Any:
    return tool(**tool_args)


def run_step(messages, model="gpt-4o-mini", **kwargs):
    messages = messages.copy()
    response = completion(model=model, messages=messages, response_format={"type": "json_object"}, **kwargs)
    response_message = response.choices[0].message.model_dump()
    messages.append(response_message)
    assistant_json = json.loads(response_message.get("content", ""))
    if "ACTION" in assistant_json:
        console_print_react_tool_action_inputs(assistant_json)
        tool_name = assistant_json["ACTION"]["tool_name"]
        tool_result = call_tool(TOOL_LKP[tool_name], assistant_json["ACTION"]["tool_arguments"])
        console_print_react_tool_action_outputs(tool_name, tool_result)
        if tool_name == "final_answer":
            return messages
        else:
            messages.append(
                {
                    "role": "user",
                    "content": "OBSERVATION:\n" + str(tool_result),
                }
            )
    else:
        messages.append(
            {
                "role": "user",
                "content": 'Remember to always respond in structured JSON format with the fields "THOUGHT" and "ACTION". Please try again.',
            }
        )
    return messages


def react_loop(task: str, model="gpt-4o-mini", max_steps=10, **kwargs):
    messages = [
        {"role": "system", "content": REACT_SYSTEM_PROMPT},
        {"role": "user", "content": task},
    ]
    console_print_user_request(messages, model)
    done_calling_tools = False
    for counter in range(max_steps):
        done_calling_tools = messages[-1]["role"] == "assistant" and "final_answer" in messages[-1].get("content")
        if done_calling_tools:
            break
        messages = run_step(messages, model=model, **kwargs)
    return messages

```

## Example Task 1
We will attempt to solve the same task as before using the ReAct prompting technique
and the same model as before.

### gpt-4o-mini

In [21]:
from react import react_loop

In [22]:
messages_final = react_loop(task)

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│         Recently on Jan 2 2025, Steph Curry made a series of 3 pointers in one game without missing.            │
│         How many three pointers did he make in total that game?                                                 │
│         How many points did he score in total that game?                                                        │
│         How many combined points did both teams score on that game?                                             │
│         Of the total points scored by both teams, what percentage was made by Steph Curry?                      │
│                                                                                                                 │
│         One more task. Lebron James also played a game on Jan 2 2025.                                           │
│         How old is Lebron James and how many points did he score in his game on Jan 2 2025?                     │
│         Take his total points scored that game and raise it to the power of 5. What is the result?              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─ gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── THOUGHT and ACTION ───────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── THOUGHT ──────────────────────────────────────────────────╮ │
│ │ To answer these questions, I need to search for news related to Steph Curry's performance on January 2,     │ │
│ │ 2025, that includes the number of three-pointers made, total points scored, and the combined points of both │ │
│ │ teams in that game. After gathering this information, I can calculate the percentage of points scored by    │ │
│ │ Steph Curry. Then, I will also search for LeBron James' performance in the same game, including his age and │ │
│ │ points scored, before performing the mathematical operation. I'll start with the search for Steph Curry's   │ │
│ │ game stats.                                                                                                 │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── ACTION ───────────────────────────────────────────────────╮ │
│ │ {'query': 'Steph Curry January 2 2025 game stats three pointers total points combined score'}               │ │
│ ╰─ web_search ────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── OBSERVATION ──────────────────────────────────────────────────╮
│ Result 1:                                                                                                       │
│ Title: Steph Curry Stats In January 2025 - StatMuse                                                             │
│ URL: https://www.statmuse.com/nba/ask/steph-curry-stats-in-january-2025                                         │
│ Stephen Curry has averaged 29.0 points, 6.7 rebounds and 3.3 assists in 3 games this month.                     │
│                                                                                                                 │
│ Result 2:                                                                                                       │
│ Title: Stephen Curry 2025 Game Log - StatMuse                                                                   │
│ URL: https://www.statmuse.com/nba/player/stephen-curry-787/game-log?seasonYear=2025                             │
│ Stephen Curry's game log for the 2025 NBA regular season and playoffs. Toggle Theme. Sign in/up. Home. NFL ;    │
│ CFB ; NBA ; FC ; NHL ; ... Stephen Curry ...                                                                    │
│                                                                                                                 │
│ Result 3:                                                                                                       │
│ Title: Stephen Curry 2024-25 Full Game Log - NBA Players Stats                                                  │
│ URL: https://www.landofbasketball.com/nba_players_game_logs/2025/stephen_curry_full.htm                         │
│ Game by game statistics of Stephen Curry in the 2025 NBA Season and Playoffs. ... Points - Rebounds - Assists - │
│ Steals - Blocks - 3-Pointers / Triple-...                                                                       │
│                                                                                                                 │
│ Result 4:                                                                                                       │
│ Title: Stephen Curry 2024-25 Basic Game Log - NBA Players Stats                                                 │
│ URL: https://www.landofbasketball.com/nba_players_game_logs/2025/stephen_curry.htm                              │
│ Game by game statistics of Stephen Curry in the 2025 NBA Season and Playoffs. His Basic numbers including       │
│ points, rebounds, assists, steals and blocks...                                                                 │
│                                                                                                                 │
│ Result 5:                                                                                                       │
│ Title: Stephen Curry 2024-25 Stats per Game - NBA - ESPN                                                        │
│ URL: https://www.espn.com/nba/player/gamelog/_/id/3975/stephen-curry                                            │
│ The 2024-25 NBA season stats per game for Stephen Curry of the Golden State Warriors on ESPN. Includes full     │
│ stats, per opponent, for regular and posts...                                                                   │
│                                                                                                                 │
╰─ web_search ────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── THOUGHT and ACTION ───────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── THOUGHT ──────────────────────────────────────────────────╮ │
│ │ The search results include several links to Steph Curry's stats for January 2025, but I don't see specific  │ │
│ │ details about his performance on January 2, 2025, including the number of three-pointers made, total points │ │
│ │ scored, or the combined score of both teams. Thus, I will visit the most promising link that is likely to   │ │
│ │ contain detailed game logs for that date.                                                                   │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── ACTION ───────────────────────────────────────────────────╮ │
│ │ {'url': 'https://www.statmuse.com/nba/player/stephen-curry-787/game-log?seasonYear=2025'}                   │ │
│ ╰─ visit_web_page ────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── OBSERVATION ──────────────────────────────────────────────────╮
│                                                                                                                 │
│ Stephen Curry 2025 Game Log | StatMuse                                                                          │
│                              Toggle Theme          [Sign in/up](/auth/signup)                                   │
│                                                                                                                 │
│ * [Home](/)                                                                                                     │
│  * [NFL](/nfl)                                                                                                  │
│ * [CFB](/cfb)                                                                                                   │
│ * [NBA](/nba)                                                                                                   │
│ * [FC](/fc)                                                                                                     │
│ * [NHL](/nhl)                                                                                                   │
│ * [MLB](/mlb)                                                                                                   │
│ * [PGA](/pga)                                                                                                   │
│ * [Money](/money)                                                                                               │
│  * [Scores](/scores)                                                                                            │
│ * [Trending](/trending)                                                                                         │
│  * [unde...<truncated>                                                                                          │
│                                                                                                                 │
╰─ visit_web_page ────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── OBSERVATION ──────────────────────────────────────────────────╮
│ Result 1:                                                                                                       │
│ Title: Lakers 114-106 Trail Blazers (Jan 2, 2025) Game Recap - ESPN                                             │
│ URL: https://www.espn.com/nba/recap/_/gameId/401705034                                                          │
│ LeBron goes off for 38 points, 7 treys in Lakers' win LeBron James drops 38 points with seven 3-pointers to     │
│ help propel the Lakers past the Trail Blaz...                                                                   │
│                                                                                                                 │
│ Result 2:                                                                                                       │
│ Title: LeBron James 2024-25 Stats per Game - NBA - ESPN                                                         │
│ URL: https://www.espn.com/nba/player/gamelog/_/id/1966/lebron-james                                             │
│ The 2024-25 NBA season stats per game for LeBron James of the Los Angeles Lakers on ESPN. Includes full stats,  │
│ per opponent, for regular and postseaso...                                                                      │
│                                                                                                                 │
│ Result 3:                                                                                                       │
│ Title: Lakers 114-106 Trail Blazers (Jan 2, 2025) Box Score - ESPN                                              │
│ URL: https://www.espn.com/nba/boxscore/_/gameId/401705034                                                       │
│ Box score for the Los Angeles Lakers vs. Portland Trail Blazers NBA game from January 2, 2025 on ESPN. Includes │
│ all points, rebounds and steals stats.                                                                          │
│                                                                                                                 │
│ Result 4:                                                                                                       │
│ Title: LeBron James 2025 Game Log | StatMuse                                                                    │
│ URL: https://www.statmuse.com/nba/player/lebron-james-1780/game-log?seasonYear=2025                             │
│ LeBron James's game log for the 2025 NBA regular season and playoffs                                            │
│                                                                                                                 │
│ Result 5:                                                                                                       │
│ Title: Lakers vs Trail Blazers Final Score: LeBron, Christie power way to win ...                               │
│ URL:                                                                                                            │
│ https://www.silverscreenandroll.com/2025/1/2/24334743/lakers-vs-trail-blazers-final-score-recap-stats-box-score │
│ -lebron-james-max-christie-austin-reaves                                                                        │
│ LeBron James scored 18 in the first half and ended the game with 38 points on 15-25 shooting with seven         │
│ 3-pointers.                                                                                                     │
│                                                                                                                 │
╰─ web_search ────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── THOUGHT and ACTION ───────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── THOUGHT ──────────────────────────────────────────────────╮ │
│ │ The search results show that LeBron James scored 38 points in the game against the Trail Blazers on January │ │
│ │ 2, 2025. Additionally, I need to calculate LeBron's age on that date. He was born on December 30, 1984.     │ │
│ │ Therefore, on January 2, 2025, he would be 40 years old. I will now raise the number of points he scored    │ │
│ │ (38) to the power of 5 to get the final result.                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── ACTION ───────────────────────────────────────────────────╮ │
│ │   1 lebron_points = 38                                                                                      │ │
│ │   2 lebron_age = 2025 - 1984                                                                                │ │
│ │   3 result_power_5 = lebron_points ** 5                                                                     │ │
│ │   4 lebron_age, result_power_5                                                                              │ │
│ ╰─ execute_python_code ───────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── OBSERVATION ──────────────────────────────────────────────────╮
│ stdout:                                                                                                         │
│ Out[1]: (41, 79235168)                                                                                          │
│                                                                                                                 │
│ is_success: True                                                                                                │
╰─ execute_python_code ───────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── THOUGHT and ACTION ───────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── THOUGHT ──────────────────────────────────────────────────╮ │
│ │ LeBron James is 41 years old and, during the game on January 2, 2025, he scored 38 points, which when       │ │
│ │ raised to the power of 5 gives a result of 79,235,168. Now I will compile all the information and present   │ │
│ │ the final answers for both Steph Curry and LeBron James.                                                    │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── ACTION ───────────────────────────────────────────────────╮ │
│ │ {'answer': 'On January 2, 2025, Steph Curry made 8 three-pointers and scored a total of 30 points,          │ │
│ │ accounting for approximately 12.30% of the combined score of 244 points in his game against the 76ers.      │ │
│ │ LeBron James, who is 41 years old, scored 38 points in his game against the Trail Blazers, and the result   │ │
│ │ of raising his points scored to the power of 5 is 79,235,168.'}                                             │ │
│ ╰─ final_answer ──────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── OBSERVATION ──────────────────────────────────────────────────╮
│ On January 2, 2025, Steph Curry made 8 three-pointers and scored a total of 30 points, accounting for           │
│ approximately 12.30% of the combined score of 244 points in his game against the 76ers. LeBron James, who is 41 │
│ years old, scored 38 points in his game against the Trail Blazers, and the result of raising his points scored  │
│ to the power of 5 is 79,235,168.                                                                                │
╰─ final_answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [23]:
eval_example_one(messages_final[-1]["content"])

{'correct': False,
 'explanation': "The LLM incorrectly stated LeBron James's age. According to the ground truth, LeBron was 40 years old on January 2, 2025, not 41. All other aspects of the LLM's answer, including Steph Curry's performance and LeBron's points scored and calculation, were accurate. However, since the age was incorrect, the LLM did not get everything correct in its predicted answer."}

# Coding Action Agent

We have just utilized the standard JSON tool calling approach. This is a common approach used by the LLM APIs from OpenAI, Anthropic, Google, etc.
The actions are tool calls consisting of JSON objects which state the function and arguments to use. Another approach is
getting the LLMs to call the tools within code. I had heard of this before but read more about it 
in the [smolagents](https://huggingface.co/blog/smolagents#code-agents) blog post. One good paper on this topic
is [Executable Code Actions Elicit Better LLM Agents](https://huggingface.co/papers/2402.01030). Here
is an image from the paper illustrating the differences between the JSON tool approach and code approach:

![[Figure from the CodeAct Paper](https://arxiv.org/pdf/2402.01030)](imgs/code_act.png)

 Instead of generating static JSON objects to represent tool calls, the code approach allows LLMs to write and execute Python code. This makes tool interactions more dynamic and adaptable, as the LLM can handle logic, conditionals, and iterations directly within the generated code. This flexibility enhances how LLMs can interact with complex tasks and environments.

Since we are relying on the LLM to write much more code, it's even more important to have a proper sandbox environment.
Before we get to the code agent implementation, let's first take a detour to see how we can create a sandbox environment for executing arbitrary python code.

## Modal Sandbox Environment - ` IPython` REPL

[Modal Sandboxes](https://modal.com/docs/guide/sandbox) are super cool! I'm still learning about them, but they are a great way to execute arbitrary code in a secure environment. I wanted to build a simple proof of concept `ipython` REPL within an isolated sandbox environment.

The Modal sandbox implementation creates a secure environment for executing arbitrary Python code while maintaining state between executions. Let's break down how it works:

1. **Custom IPython Shell**: We create a persistent IPython shell that runs in a Modal container/sandbox, allowing us to maintain state and execute code interactively. This gives us the familiar IPython REPL experience but in a secure, isolated environment.

2. **Input/Output Communication**: I use a simple JSON-based protocol to communicate between the local environment and the Modal container. Code is sent to the container for execution, and results (including stdout, stderr, and the last expression value) are returned in a structured format.

3. **State Persistence**: Unlike typical serverless functions that are stateless, this sandbox maintains state between executions when using the same sandbox instance. This means variables and imports persist across multiple code executions.

Using Modal's sandbox provides security. This makes it safe to execute arbitrary Python code without risking the host system's security. The sandbox is particularly useful for AI agents that need to execute Python code as part of their reasoning process, as it provides a secure environment for code execution while maintaining the interactive nature of an IPython REPL.

Here is the code for my proof of concept IPython REPL within a Modal sandbox:

In [24]:
# | echo: false
file_path = "python_sandbox.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
import json

import modal

# Create image with IPython installed
image = modal.Image.debian_slim().pip_install("ipython", "pandas")


# Create the driver program that will run in the sandbox
def create_driver_program():
    return """
import json
import sys
import re
from IPython.core.interactiveshell import InteractiveShell
from IPython.utils.io import capture_output

def strip_ansi_codes(text):
    ansi_escape = re.compile(r'\\x1B(?:[@-Z\\\\-_]|\\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

# Create a persistent IPython shell instance
shell = InteractiveShell()
shell.colors = 'NoColor'  # Disable color output
shell.autoindent = False  # Disable autoindent

# Keep reading commands from stdin
while True:
    try:
        # Read a line of JSON from stdin
        command = json.loads(input())
        code = command.get('code')
        
        if code is None:
            print(json.dumps({"error": "No code provided"}))
            continue
            
        # Execute the code and capture output
        with capture_output() as captured:
            result = shell.run_cell(code)

        # Clean the outputs
        stdout = strip_ansi_codes(captured.stdout)
        stderr = strip_ansi_codes(captured.stderr)
        error = strip_ansi_codes(str(result.error_in_exec)) if not result.success else None

        # Format the response
        response = {
            "stdout": stdout,
            "stderr": stderr,
            "success": result.success,
            "result": repr(result.result) if result.success else None,
            "error": error
        }
        
        # Send the response
        print(json.dumps(response), flush=True)
        
    except Exception as e:
        print(json.dumps({"error": strip_ansi_codes(str(e))}), flush=True)
"""


def create_sandbox():
    """Creates and returns a Modal sandbox running an IPython shell."""
    app = modal.App.lookup("ipython-sandbox", create_if_missing=True)

    # Create the sandbox with the driver program
    with modal.enable_output():
        sandbox = modal.Sandbox.create("python", "-c", create_driver_program(), image=image, app=app)

    return sandbox


def execute_python_code(code: str, sandbox=None) -> dict:
    created_sandbox = False
    if sandbox is None:
        sandbox = create_sandbox()
        created_sandbox = True
    # Send the code to the sandbox
    sandbox.stdin.write(json.dumps({"code": code}))
    sandbox.stdin.write("\n")
    sandbox.stdin.drain()

    # Get the response
    response = next(iter(sandbox.stdout))
    if created_sandbox:
        sandbox.terminate()
    return json.loads(response)

```

In [25]:
from python_sandbox import create_sandbox, execute_python_code

One simple use case is to spin up a sandbox, execute some code, and then terminate the sandbox automatically. This is what happens if you don't pass in a sandbox object.

In [26]:
code = """
print('This is a test running within a Modal Sandbox!!!')
x = 2
y = 6
print(x+y)
y-x
"""
execute_python_code(code=code)

{'stdout': 'This is a test running within a Modal Sandbox!!!\n8\nOut[1]: 4\n',
 'stderr': '',
 'success': True,
 'result': '4',
 'error': None}

Another interesting use case is to create a persistent sandbox and then use it for multiple python code executions. The state is maintained between executions.

In [27]:
sandbox = create_sandbox()
execute_python_code(code="x=2", sandbox=sandbox)

{'stdout': '', 'stderr': '', 'success': True, 'result': 'None', 'error': None}

In [28]:
execute_python_code(code="y=6; print(x+y)", sandbox=sandbox)

{'stdout': '8\n',
 'stderr': '',
 'success': True,
 'result': 'None',
 'error': None}

In [29]:
execute_python_code(code="y-x", sandbox=sandbox)

{'stdout': 'Out[1]: 4\n',
 'stderr': '',
 'success': True,
 'result': '4',
 'error': None}

In [30]:
code = """
numbers = list(range(1, 6))
squares = [n**2 for n in numbers]
sum_squares = sum(squares)
print(f"Numbers: {numbers}")
print(f"Squares: {squares}")
print(f"Sum of squares: {sum_squares}")
numbers
"""
execute_python_code(code=code, sandbox=sandbox)

{'stdout': 'Numbers: [1, 2, 3, 4, 5]\nSquares: [1, 4, 9, 16, 25]\nSum of squares: 55\nOut[1]: [1, 2, 3, 4, 5]\n',
 'stderr': '',
 'success': True,
 'result': '[1, 2, 3, 4, 5]',
 'error': None}

I can terminate the sandbox when I am done with it.

In [31]:
sandbox.terminate()

## Code Agent Implementation

Here is a proof of concept implementation of a code agent. Much like the rest of this post, this is all for educational purposes.
I got all my inspiration from the [smolagents](https://github.com/huggingface/smolagents) library.
Since their repo is small it's such a great learning resource! Go check it out if you want something more robust.

I hacked this together, and the system prompt is sort of long. 
But I hope this gives a good illustration of the basics.
It's really just the same things we have already seen. 

**LLM + system prompt + tools + sandbox python environment + for loop = code agent**




In [32]:
# | echo: false
file_path = "coding_agent.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
import re

from litellm import completion
from python_sandbox import create_sandbox, execute_python_code
from utils import (
    console_print_code_agent_assistant_message,
    console_print_code_agent_code_block,
    console_print_code_agent_observation,
    console_print_llm_output,
    console_print_step,
    console_print_user_request,
)

CODING_AGENT_SYSTEM_PROMPT = """
You are an expert Python programmer who solves problems incrementally using a secure IPython REPL environment.
You break down complex tasks into small, verifiable steps, always checking your intermediate results before proceeding.

PROBLEM-SOLVING FORMAT:
You solve tasks through a repeating cycle of three steps:

Thought: Explain your reasoning and what you expect to learn
Code: Write code to solve step by step
Observation: Review the code execution results from the user to inform next steps

This cycle repeats, with each iteration building on previous results, until the task is completed. 
The task is only complete when you have gathered all the information you need to solve the problem.
You then submit your final answer to the user with a "FINAL ANSWER" submission tag.

You do the thinking and generate thoughts.
You write the code.
The user will execute the code and provide you the output/observation to inform your next steps.

ENVIRONMENT CAPABILITIES:
1. Secure Sandbox:
   - Isolated sandbox container for safe arbitrary code execution
   - Persistent state between executions
   - Nothing can go wrong on the host machine. Install any packages you need and run any code you need.
   - Built with Modal and IPython for secure code execution

2. Pre-imported Tools (Feel free to use these tools as needed or create your own from scratch!)
   - web_search(query: str) - Search the web for the given query. Always print the results.
   - visit_web_page(url: str) - Visit and extract content from the given URL. Always print the results.

3. String Formatting Requirements:
   - All print statements must use double backslashes for escape characters
   - Example: print("\\nHello") instead of print("\nHello")
   - This applies to all string literals containing \n, \r, \t etc.
   - This is required to prevent string termination errors in the sandbox

4. Code Execution Response Format:
   {
     'stdout': str,  # Printed output
     'stderr': str,  # Error messages
     'success': bool,  # Execution success
     'result': str,  # Last expression value
     'error': str | None  # Exception details
   }
   
PROBLEM-SOLVING APPROACH:
1. Break down the problem into smallest possible steps
2. Write and test one small piece at a time
3. Verify each intermediate result before continuing
4. Build upon working code incrementally
5. Keep track of what you learn at each step


**IMPORTANT:** 
    - Follow the Code formatting exactly as shown in the examples.
    - After the final code execution and observation, you must provide a "FINAL ANSWER" submission tag.
        This is the only message you do not need to use code for.
        Submit your FINAL ANSWER using markdown and use all the information you have gathered 
        to give a comprehensive answer to the user.

Your reward comes from solving tasks reliably and delighting users with clear, well-validated code developed step-by-step.

EXAMPLES:

Example 1
Task: "Find the average word length in a sentence, excluding punctuation."

Thought: Let's first get our sentence and print it to verify what we're working with.

Code:
```python
text = "Hello, World! This is a test sentence."
print(f"Working with text: {text}")
```<end_code>

Observation: {
    'stdout': 'Working with text: Hello, World! This is a test sentence.\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
 }

Thought: Now let's remove punctuation and verify our cleaning worked.

Code:
```python
import string
cleaned_text = text.translate(str.maketrans("", "", string.punctuation))
print(f"Cleaned text: {cleaned_text}")
```<end_code>

Observation: {
    'stdout': 'Cleaned text: Hello World This is a test sentence\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Let's split into words and verify our word list.

Code:
```python
words = cleaned_text.split()
print(f"Words: {words}")
print(f"Number of words: {len(words)}")
```<end_code>

Observation: {
    'stdout': "Words: ['Hello', 'World', 'This', 'is', 'a', 'test', 'sentence']\nNumber of words: 7\n",
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Now we can calculate word lengths and verify them individually.

Code:
```python
word_lengths = [len(word) for word in words]
print("Length of each word:")
for word, length in zip(words, word_lengths):
    print(f"'{word}': {length} characters")
```<end_code>

Observation: {
    'stdout': "Length of each word:\n'Hello': 5 characters\n'World': 5 characters\n'This': 4 characters\n'is': 2 characters\n'a': 1 characters\n'test': 4 characters\n'sentence': 8 characters\n",
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
    }

Thought: Finally, calculate the average and format it nicely.

Code:
```python
average_length = sum(word_lengths) / len(word_lengths)
print(f"Average word length: {average_length:.2f} characters")
```<end_code>


Observation: {
    stdout': 'Average word length: 4.14 characters\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: I have all the information I need to solve the users request. The answer is 4.14 characters.

FINAL ANSWER:
The average word length in a sentence, excluding punctuation, is 4.14 characters.


Example 2
Task: "Analyze a dataset of movie ratings. Calculate average ratings by genre, identify the most popular genres, and create a summary of the top-rated movies."

Thought: First, let's try to use pandas for this analysis.

Code:
```python
import pandas as pd
```<end_code>

Observation: {
    'stdout': '',
    'stderr': 'ModuleNotFoundError: No module named "pandas"',
    'success': False,
    'error': 'ModuleNotFoundError: No module named "pandas"'
}

Thought: We need to install pandas first. Let's do that and verify the installation.

Code:
```python
!pip install pandas
import pandas as pd
print("Pandas successfully imported!")
```<end_code>

Observation: {
    'stdout': 'Successfully installed pandas-2.1.0\nPandas successfully imported!\n',
    'stderr': '',
    'success': True,
    'result': None,
    'error': None
}

Thought: Let's start by loading a small sample of data and verify its structure.

Code:
```python
# Create sample data to work with
data = {
    'title': ['The Matrix', 'Inception', 'Jurassic Park', 'The Notebook', 'Die Hard'],
    'genre': ['Sci-Fi,Action', 'Sci-Fi,Thriller', 'Action,Adventure', 'Romance,Drama', 'Action,Thriller'],
    'rating': [8.7, 8.8, 8.1, 7.8, 8.2]
}
df = pd.DataFrame(data)
print("Data sample:\\n")
print(df.head())
```<end_code>

Observation: {
    'stdout': 'Data sample:\n           title             genre  rating\n0     The Matrix     Sci-Fi,Action     8.7\n1      Inception   Sci-Fi,Thriller     8.8\n2  Jurassic Park  Action,Adventure     8.1\n3   The Notebook     Romance,Drama     7.8\n4       Die Hard   Action,Thriller     8.2\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: We need to split the genre column since movies can have multiple genres. Let's do this step by step.

Code:
```python
# First, let's see what unique genres we have
all_genres = set()
for genres in df['genre']:
    all_genres.update(genres.split(','))
print("Unique genres:", sorted(all_genres))
```<end_code>

Observation: {
    'stdout': "Unique genres: ['Action', 'Adventure', 'Drama', 'Romance', 'Sci-Fi', 'Thriller']\n",
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Now let's create a function to analyze ratings by genre and test it with one genre.

Code:
```python
def get_genre_stats(genre):
    # Get movies in this genre
    genre_movies = df[df['genre'].str.contains(genre)]
    avg_rating = genre_movies['rating'].mean()
    count = len(genre_movies)
    return {'genre': genre, 'avg_rating': avg_rating, 'movie_count': count}

# Test with one genre
action_stats = get_genre_stats('Action')
print("Action genre stats:", action_stats)
```<end_code>

Observation: {
    'stdout': "Action genre stats: {'genre': 'Action', 'avg_rating': 8.333333333333332, 'movie_count': 3}\n",
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: The function works! Let's analyze all genres and create a summary DataFrame.

Code:
```python
# Create stats for all genres
genre_stats = [get_genre_stats(genre) for genre in all_genres]
stats_df = pd.DataFrame(genre_stats)
stats_df = stats_df.sort_values('avg_rating', ascending=False)
print("\\nGenre Statistics:")
print(stats_df)
```<end_code>

Observation: {
    'stdout': '\nGenre Statistics:\n       genre  avg_rating  movie_count\n0     Sci-Fi    8.750000            2\n4   Thriller    8.500000            2\n2     Action    8.333333            3\n1  Adventure    8.100000            1\n3    Romance    7.800000            1\n5      Drama    7.800000            1\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Finally, let's create a comprehensive summary including top movies in each genre.

Code:
```python
def create_summary():
    summary = "Movie Analysis Summary:\\n\\n"
    summary += "Top Genres by Average Rating:\\n"
    for _, row in stats_df.head(3).iterrows():
        summary += f"- {row['genre']}: {row['avg_rating']:.1f} ({row['movie_count']} movies)\\n"
    
    summary += "\\nTop Rated Movie Overall:\\n"
    top_movie = df.loc[df['rating'].idxmax()]
    summary += f"- {top_movie['title']} ({top_movie['rating']})"
    
    print(summary)
    return summary

print(create_summary())
```<end_code>

Observation: { 
    'stdout': 'Movie Analysis Summary:\n\nTop Genres by Average Rating:\n- Sci-Fi: 8.8 (2 movies)\n- Thriller: 8.5 (2 movies)\n- Action: 8.3 (3 movies)\n\nTop Rated Movie Overall:\n- Inception (8.8)\nMovie Analysis Summary:\n\nTop Genres by Average Rating:\n- Sci-Fi: 8.8 (2 movies)\n- Thriller: 8.5 (2 movies)\n- Action: 8.3 (3 movies)\n\nTop Rated Movie Overall:\n- Inception (8.8)\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: I now can format the answer into markdown and provide a comprehensive summary.

FINAL ANSWER:
# Movie Analysis Summary

## Top Genres by Average Rating
* **Sci-Fi**: 8.8 (2 movies)
* **Thriller**: 8.5 (2 movies)
* **Action**: 8.3 (3 movies)

## Top Rated Movie Overall
* **Inception** (8.8)

Example 3
Task: "Analyze the 30-day volatility trends of tech giants (Apple, Microsoft, Google) over the past year and identify which company has been the most volatile"

Thought: First attempt to get historical stock data directly.

Code:
```python
import yfinance
tickers = ['AAPL', 'MSFT', 'GOOGL']
data = yfinance.download(tickers, period='1y')
print("Downloaded data preview:")
print(data.head(2))
```<end_code>

Observation: {
    'stdout': "---------------------------------------------------------------------------\nModuleNotFoundError                       Traceback (most recent call last)\nFile <ipython-input-1-3a34c6ac3ab5>:1\n----> 1 import yfinance\n      2 tickers = ['AAPL', 'MSFT', 'GOOGL']\n      3 data = yfinance.download(tickers, period='1y')\n\nModuleNotFoundError: No module named 'yfinance'\n",
    'stderr': '',
    'success': False,
    'result': None,
    'error': "No module named 'yfinance'"
}

Thought: We need to install yfinance first. Let's do that and verify the installation.

Code:
```python
!pip install yfinance
import yfinance as yf
print("Successfully installed and imported yfinance!")
```<end_code>

Observation: {
    'stdout': 'Collecting yfinance\r\n  Downloading yfinance-0.2.52-py2.py3-none-any.whl.metadata (5.8 kB)\r\nRequirement already satisfied: pandas>=1.3.0 ......\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Now let's download the data and analyze the available columns.

Code:
```python
import pandas as pd
tickers = ['AAPL', 'MSFT', 'GOOGL']
data = yf.download(tickers, period='1y')
print("Data shape:", data.shape)
print("\\nColumns available:", data.columns.levels[0].tolist())
```<end_code>

Observation: {
    'stdout': "Data shape: (252, 15)\n\nColumns available: ['Close', 'High', 'Low', 'Open', 'Volume']\n",
    'stderr': '\r[                       0%                       ]\r[**********************67%*******                ]  2 of 3 completed\r[*********************100%***********************]  3 of 3 completed\n',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Let's check the data structure and calculate 30-day rolling volatility using closing prices.

Code:
```python
# Calculate daily returns and 30-day rolling volatility
returns = data['Close'].pct_change()
volatility = returns.rolling(window=30).std() * (252 ** 0.5)  # Annualized volatility
print("\\nFirst few days of volatility (will be NaN during first 30 days due to rolling window):")
print(volatility.head())
print("\\nLast 5 days of volatility:")
print(volatility.tail())
```<end_code>

Observation: {
    'stdout': '\nFirst few days of volatility (will be NaN during first 30 days due to rolling window):\nTicker      AAPL  GOOGL  MSFT\nDate                         \n2024-01-18   NaN    NaN   NaN\n2024-01-19   NaN    NaN   NaN\n2024-01-22   NaN    NaN   NaN\n2024-01-23   NaN    NaN   NaN\n2024-01-24   NaN    NaN   NaN\n\nLast 5 days of volatility:\nTicker          AAPL     GOOGL      MSFT\nDate                                    \n2025-01-13  0.184242  0.316788  0.184272\n2025-01-14  0.184753  0.318345  0.181594\n2025-01-15  0.191293  0.327256  0.196739\n2025-01-16  0.222245  0.330185  0.189958\n2025-01-17  0.219824  0.331567  0.192567\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: Now let's create a final summary comparing the volatility of each stock and identify the most volatile one.

Code:
```python
# Calculate mean volatility for each stock (excluding NaN values)
avg_vol = volatility.mean()
max_vol = volatility.max()
most_volatile = avg_vol.idxmax()

summary = {
    'most_volatile_stock': most_volatile,
    'average_volatility': {
        'AAPL': f"{avg_vol['AAPL']:.2%}",
        'MSFT': f"{avg_vol['MSFT']:.2%}",
        'GOOGL': f"{avg_vol['GOOGL']:.2%}"
    },
    'peak_volatility': {
        'AAPL': f"{max_vol['AAPL']:.2%}",
        'MSFT': f"{max_vol['MSFT']:.2%}",
        'GOOGL': f"{max_vol['GOOGL']:.2%}"
    },
    'analysis_period': f"{data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}"
}

print("\\nVolatility Analysis Summary:")
print(f"Most volatile stock: {summary['most_volatile_stock']}")
print("\\nAverage Volatility:")
for stock, vol in summary['average_volatility'].items():
    print(f"{stock}: {vol}")
print("\\nPeak Volatility:")
for stock, vol in summary['peak_volatility'].items():
    print(f"{stock}: {vol}")
print(f"\\nAnalysis Period: {summary['analysis_period']}")
```<end_code>

Observation: {
    'stdout': '\nVolatility Analysis Summary:\nMost volatile stock: GOOGL\n\nAverage Volatility:\nAAPL: 22.47%\nMSFT: 20.09%\nGOOGL: 27.26%\n\nPeak Volatility:\nAAPL: 32.75%\nMSFT: 25.31%\nGOOGL: 39.13%\n\nAnalysis Period: 2024-01-22 to 2025-01-21\n',
    'stderr': '',
    'success': True,
    'result': 'None',
    'error': None
}

Thought: I now can format the answer into markdown and provide a comprehensive summary.

FINAL ANSWER:
# Volatility Analysis Summary

**Analysis Period:** 2024-01-22 to 2025-01-21
**Most Volatile Stock:** GOOGL

## Average Volatility
* **AAPL**: 22.47%
* **MSFT**: 20.09%
* **GOOGL**: 27.26%

## Peak Volatility
* **AAPL**: 32.75%
* **MSFT**: 25.31%
* **GOOGL**: 39.13%


Example 4
Task: "Give a detailed summary of Trumps 2025 Inauguration speech. Consult multiple sources."

Thought: To provide a detailed summary of Trump's 2025 Inauguration speech, I will first search for the text of the speech and analyze multiple sources for their interpretations and key points. This will help ensure a comprehensive summary. 

Code:
```python
# Initiating a web search for Trump's 2025 inauguration speech and summaries
web_results = web_search("Trump 2025 inauguration speech summary")
print(web_results)
```<end_code>

Observation: {
'stdout': '[{\'title\': "Trump\'s inaugural address, annotated and fact-checked", \'href\': \'https://www.cnn.com/interactive/politics/2025/donald-trump-inaugural-speech-dg/\', \'body\': "Read a line-by-line analysis of President Donald Trump\'s inaugural address. ... 2025, is liberation day. This is wartime language. Trump suggests Americans have been under siege until today."}, {\'title\': "4 takeaways from Trump\'s second inaugural address : NPR", \'href\': \'https://www.npr.org/2025/01/20/g-s1-43759/trump-inaugural-address-key-moments\', \'body\': "For updates, context and analysis of Donald Trump\'s 2025 inauguration, check out NPR\'s live blog throughout the day Monday."}, {\'title\': "Takeaways from Trump\'s second inaugural address | AP News", \'href\': \'https://apnews.com/article/trump-inaugural-address-takeaways-212571b9f231469010c31e06ec530199\', \'body\': \'NEW YORK (AP) — President Donald Trump\\\'s second inaugural address featured similar themes to his first: a sweeping indictment of the country he inherits and grand promises to fix its problems.. Eight years ago, Trump described "American carnage" and promised to end it immediately. On Monday, he declared that the country\\\'s "decline" will end immediately, ushering in "the golden ...\'}, {\'title\': "6 takeaways from Trump\'s inaugural address - CBS News", \'href\': \'https://www.cbsnews.com/news/6-takeaways-from-trumps-inaugural-address/\', \'body\': \'Here are the highlights from Mr. Trump\\\'s second inaugural address. "The golden age of America begins right now" ... See the Inauguration Day schedule for Trump\\\'s 2025 presidential swearing in.\'}, {\'title\': "Donald Trump\'s Second Inaugural Speech, Annotated", \'href\': \'https://www.nytimes.com/interactive/2025/01/20/us/trump-inauguration-speech-annotated.html\', \'body\': \'1. Trump promised an "America First" vision in his first term, and he made a similar promise in his first inaugural address. This idea has been one of the animating ideas behind his political ...\'}]\n',
 'stderr': '',
 'success': True,
 'result': 'None',
 'error': None
}

Thought: I found several articles summarizing Trump's 2025 Inauguration speech, including detailed analyses and key takeaways. The notable sources include CNN, NPR, AP News, CBS News, and The New York Times. 
I'll extract information from these sources to compile a comprehensive summary. 
First I will visit each webpage to get the full details.

Code:
```python
for source in web_results:
    print(f"Visiting {source['title']} at {source['href']}")
    print(visit_web_page(source['href']))
```<end_code>

Observation: {
'stdout': "Visiting Trump's inaugural address, annotated and fact-checked at https://www.cnn.com/interactive/politics/2025/donald-trump-inaugural-speech-dg/\nDonald Trump's second inaugural speech, annotated and fact checked - CNN                            \n\n... (content truncated)\nVisiting 4 takeaways from Trump's second inaugural address : NPR at https://www.npr.org/2025/01/20/g-s1-43759/trump-inaugural-address-key-moments\n4 takeaways from Trump’s second inaugural address : NPR\n\nAccessibility links\n\n* [Skip to main conten\n\n... (content truncated)\nVisiting Takeaways from Trump's second inaugural address | AP News at https://apnews.com/article/trump-inaugural-address-takeaways-212571b9f231469010c31e06ec530199\nKey moments from Trump's second inaugural address | AP News\n\nMenu\n[![AP Logo](https://assets.apnews.\n\n... (content truncated)\nVisiting 6 takeaways from Trump's inaugural address - CBS News at https://www.cbsnews.com/news/6-takeaways-from-trumps-inaugural-address/\n6 takeaways from Trump's inaugural address - CBS News\n\n* [Latest](/)\n  + [U.S.](/us/)\n  + [Inaugurat\n\n... (content truncated)\nVisiting Donald Trump's Second Inaugural Speech, Annotated at https://www.nytimes.com/interactive/2025/01/20/us/trump-inauguration-speech-annotated.html\nDonald Trump’s Second Inaugural Speech, Annotated - The New York Times\n  \n\n[Skip to content](#site-c\n\n... (content truncated)\n",
 'stderr': '',
 'success': True,
 'result': 'None',
 'error': None
 }       

Thought: I have collect much information from multiple sources. I now can format the answer into markdown and provide a comprehensive summary while also linking to original sources.

FINAL ANSWER:
# Trump's 2025 Inauguration Speech Summary
- Trump talked about .... 

"""


def extract_code_blocks(response_text: str) -> list[str]:
    # Pattern matches content between ```python and <end_code>, ignoring "Code:" prefix
    pattern = r"```python\n(.*?)<end_code>"
    # re.DOTALL allows . to match newlines
    matches = re.findall(pattern, response_text, re.DOTALL)
    # Clean up any "Code:" prefix, backticks, and whitespace
    return [block.replace("Code:", "").replace("```", "").strip() for block in matches]


def code_agent(task: str, model: str = "gpt-4o-mini", max_iterations: int = 20):
    sb = create_sandbox()

    # Copy the existing tools.py into the sandbox
    with open("web_tools.py", "r") as source_file:
        tools_content = source_file.read()

    with sb.open("web_tools.py", "w") as sandbox_file:
        sandbox_file.write(tools_content)

    execute_python_code("!pip install requests markdownify duckduckgo-search", sb)
    execute_python_code("import requests; from web_tools import web_search, visit_web_page;", sb)

    messages = [{"role": "system", "content": CODING_AGENT_SYSTEM_PROMPT}, {"role": "user", "content": task}]
    console_print_user_request(messages, model)
    for i in range(max_iterations):
        console_print_step(i)
        response = completion(model="gpt-4o-mini", messages=messages, stop=["<end_code>"])
        asst_message = response.choices[0].message.content
        contains_code = "Code:" in asst_message or "```python" in asst_message or "end_code" in asst_message
        if "FINAL ANSWER" in asst_message or not contains_code:
            messages.append({"role": "assistant", "content": asst_message})
            console_print_llm_output(asst_message)
            break
        asst_message = asst_message + "<end_code>"
        console_print_code_agent_assistant_message(asst_message)
        messages.append({"role": "assistant", "content": asst_message})
        try:
            code = extract_code_blocks(messages[-1]["content"])[0]
            console_print_code_agent_code_block(code)
        except Exception:
            messages.append(
                {
                    "role": "user",
                    "content": """
                            The was an error in extracting your code snippet.
                            The code is probably correct but you did not put it between the ```python and <end_code> tags.
                            Like this:
                                Code:
                                ```python
                                ...
                                ```<end_code>
                            Please attempt the same code again.
                            """,
                }
            )
            continue

        observation = execute_python_code(code, sb)
        console_print_code_agent_observation(observation)
        messages.append({"role": "user", "content": f"Observation: {observation}"})

    sb.terminate()
    return messages

```

In [33]:
from coding_agent import code_agent

messages_final = code_agent(task)

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│         Recently on Jan 2 2025, Steph Curry made a series of 3 pointers in one game without missing.            │
│         How many three pointers did he make in total that game?                                                 │
│         How many points did he score in total that game?                                                        │
│         How many combined points did both teams score on that game?                                             │
│         Of the total points scored by both teams, what percentage was made by Steph Curry?                      │
│                                                                                                                 │
│         One more task. Lebron James also played a game on Jan 2 2025.                                           │
│         How old is Lebron James and how many points did he score in his game on Jan 2 2025?                     │
│         Take his total points scored that game and raise it to the power of 5. What is the result?              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─ gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: To answer the first part about Steph Curry, I need to determine how many three-pointers he made and    │
│ calculate points based on standard scoring (3 points per shot). I also need to find out how many total points   │
│ were scored by both teams to analyze his contribution as a percentage.                                          │
│                                                                                                                 │
│ For the second part, to find LeBron James' age, I will calculate it based on his birth date (December 30,       │
│ 1984). After determining his current age and the points he scored in the January 2 game, I will raise that      │
│ score to the fifth power.                                                                                       │
│                                                                                                                 │
│ I will start with a search for the details of the game on January 2, 2025, where Steph Curry played, along with │
│ his total points and the combined score of both teams.                                                          │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ # Search for details about the game on January 2, 2025, involving Steph Curry                                   │
│ game_search_results = web_search("Steph Curry January 2 2025 game details points scored")                       │
│ print(game_search_results)                                                                                      │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│   1 # Search for details about the game on January 2, 2025, involving Steph Curry                               │
│   2 game_search_results = web_search("Steph Curry January 2 2025 game details points scored")                   │
│   3 print(game_search_results)                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ [{'title': 'Warriors 139-105 76ers (Jan 2, 2025) Game Recap - ESPN', 'href':                                │ │
│ │ 'https://www.espn.com/nba/recap?gameId=401705033', 'body': 'Expert recap and game analysis of the Golden    │ │
│ │ State Warriors vs. Philadelphia 76ers NBA game from January 2, 2025 on ESPN. Skip to main ... Stephen Curry │ │
│ │ scores 30 points, makes all 8 3-point ...'}, {'title': 'Stephen Curry 2024-25 Full Game Log - NBA Players   │ │
│ │ Stats', 'href': 'https://www.landofbasketball.com/nba_players_game_logs/2025/stephen_curry_full.htm', 'b    │ │
│ │ ... <truncated>...                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ None                                                                                                        │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: The search results indicate that LeBron James scored 38 points in the game against the Portland Trail  │
│ Blazers on January 2, 2025. To calculate his age, I will deduct his birth year (1984) from the current year     │
│ (2025).                                                                                                         │
│                                                                                                                 │
│ Next, I'll raise his total points scored (38) to the power of 5 as requested.                                   │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ # Calculate LeBron James' age as of January 2, 2025                                                             │
│ lebron_birth_year = 1984                                                                                        │
│ current_year = 2025                                                                                             │
│ lebron_age = current_year - lebron_birth_year                                                                   │
│                                                                                                                 │
│ # Raise his points scored (38) to the power of 5                                                                │
│ lebron_points = 38                                                                                              │
│ lebron_points_power_5 = lebron_points ** 5                                                                      │
│                                                                                                                 │
│ lebron_age, lebron_points_power_5                                                                               │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│    1 # Calculate LeBron James' age as of January 2, 2025                                                        │
│    2 lebron_birth_year = 1984                                                                                   │
│    3 current_year = 2025                                                                                        │
│    4 lebron_age = current_year - lebron_birth_year                                                              │
│    5                                                                                                            │
│    6 # Raise his points scored (38) to the power of 5                                                           │
│    7 lebron_points = 38                                                                                         │
│    8 lebron_points_power_5 = lebron_points ** 5                                                                 │
│    9                                                                                                            │
│   10 lebron_age, lebron_points_power_5                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Out[1]: (41, 79235168)                                                                                      │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ (41, 79235168)                                                                                              │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Thought: From the calculations, we found that LeBron James is 41 years old as of January 2, 2025, and raising   │
│ his total points scored (38) to the power of 5 results in 79,235,168.                                           │
│                                                                                                                 │
│ Now that I have gathered all the required information, I can summarize everything concluded about both players' │
│ performances during their respective games on January 2, 2025.                                                  │
│                                                                                                                 │
│ FINAL ANSWER:                                                                                                   │
│                                                                                                                 │
│                                        Game Summary for January 2, 2025                                         │
│                                                                                                                 │
│ Steph Curry:                                                                                                    │
│                                                                                                                 │
│  • Total three-pointers made: 8                                                                                 │
│  • Total points scored: 30                                                                                      │
│  • Combined points scored by both teams: 244                                                                    │
│  • Steph Curry's contribution percentage: approximately 12.30%                                                  │
│                                                                                                                 │
│ LeBron James:                                                                                                   │
│                                                                                                                 │
│  • Age: 41 years old                                                                                            │
│  • Points scored in the game: 38                                                                                │
│  • 38 raised to the power of 5: 79,235,168                                                                      │
╰─ final-answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [34]:
eval_example_one(messages_final[-1]["content"])

{'correct': False,
 'explanation': "The LLM gave incorrect information regarding LeBron James' age. In the correct answer, it is stated that LeBron James was 40 years old on January 2, 2025, while the LLM predicted that he was 41 years old. Additionally, the rest of the information provided about both players' performances matches the ground truth, but the error regarding LeBron's age means the LLM did not answer each part of the question correctly."}

## Example Tasks

### Characters Per Token

#### gemini/gemini-2.0-flash-exp



In [35]:
task = """How many characters on average are in an LLM token? Analyze this across different OpenAI models using the tiktoken library. Please:
Install the tiktoken library. Use a relatively long sample text with multiple paragraphs.
Analyze token lengths for various OpenAI models including:
GPT-4
GPT-3.5
GPT-4-o
GPT-4o-mini
etc.
Search the tiktoken documentation or web to find the complete list of supported models. Compare the results to understand how tokenization differs between models."""

messages_final = code_agent(task, model="gemini/gemini-2.0-flash-exp")

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many characters on average are in an LLM token? Analyze this across different OpenAI models using the       │
│ tiktoken library. Please:                                                                                       │
│ Install the tiktoken library. Use a relatively long sample text with multiple paragraphs.                       │
│ Analyze token lengths for various OpenAI models including:                                                      │
│ GPT-4                                                                                                           │
│ GPT-3.5                                                                                                         │
│ GPT-4-o                                                                                                         │
│ GPT-4o-mini                                                                                                     │
│ etc.                                                                                                            │
│ Search the tiktoken documentation or web to find the complete list of supported models. Compare the results to  │
│ understand how tokenization differs between models.                                                             │
│                                                                                                                 │
╰─ gemini/gemini-2.0-flash-exp ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: To analyze the average number of characters per token for different OpenAI models using the `tiktoken` │
│ library, I will first need to install the library and load a relatively long sample text. After ensuring the    │
│ installation is successful, I will search for the list of supported models in the `tiktoken` documentation and  │
│ analyze the token lengths across the models. This will involve encoding the sample text for each model and      │
│ computing the average characters per token.                                                                     │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ # Installing the tiktoken library                                                                               │
│ !pip install tiktoken                                                                                           │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│   1 # Installing the tiktoken library                                                                           │
│   2 !pip install tiktoken                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Collecting tiktoken                                                                                         │ │
│ │   Downloading tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)   │ │
│ │ Collecting regex>=2022.1.18 (from tiktoken)                                                                 │ │
│ │   Downloading regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)   │ │
│ │      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--                                     │ │
│ │      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 98.2 MB/s eta 0:00:00                            │ │
│ │ Requirement already satisfied: requests>=2.26.0 in /usr/local/lib/ ... <truncated>...                       │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ None                                                                                                        │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: The error occurred because the `tokens` returned from the encoding process are integers, which         │
│ represent token IDs rather than string representations of tokens. Therefore, calculating the length of a token  │
│ using `len(token.encode('utf-8'))` is causing the issue.                                                        │
│                                                                                                                 │
│ Instead, I should calculate the average number of characters in the original sample text, divided by the total  │
│ number of tokens obtained from the encoding in order to understand the characters per token. I’ll adjust the    │
│ function accordingly.                                                                                           │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ # Updated function to calculate average characters per token for a given model                                  │
│ def average_chars_per_token(model, text):                                                                       │
│     encoding = tiktoken.encoding_for_model(model)  # Get the encoding for the model                             │
│     tokens = encoding.encode(text)  # Encode the text                                                           │
│     avg_chars = len(text) / len(tokens) if tokens else 0  # Calculate the average characters per token          │
│     return avg_chars                                                                                            │
│                                                                                                                 │
│ # Analyze each model and store results                                                                          │
│ results = {}                                                                                                    │
│ for model in models:                                                                                            │
│     avg_chars = average_chars_per_token(model, sample_text)                                                     │
│     results = avg_chars                                                                                         │
│                                                                                                                 │
│ results                                                                                                         │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│    1 # Updated function to calculate average characters per token for a given model                             │
│    2 def average_chars_per_token(model, text):                                                                  │
│    3     encoding = tiktoken.encoding_for_model(model)  # Get the encoding for the model                        │
│    4     tokens = encoding.encode(text)  # Encode the text                                                      │
│    5     avg_chars = len(text) / len(tokens) if tokens else 0  # Calculate the average characters per token     │
│    6     return avg_chars                                                                                       │
│    7                                                                                                            │
│    8 # Analyze each model and store results                                                                     │
│    9 results = {}                                                                                               │
│   10 for model in models:                                                                                       │
│   11     avg_chars = average_chars_per_token(model, sample_text)                                                │
│   12     results[model] = avg_chars                                                                             │
│   13                                                                                                            │
│   14 results                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Out[1]:                                                                                                     │ │
│ │ {'gpt-4': 4.774774774774775,                                                                                │ │
│ │  'gpt-3.5-turbo': 4.774774774774775,                                                                        │ │
│ │  'gpt-4o': 4.818181818181818,                                                                               │ │
│ │  'gpt-4o-mini': 4.818181818181818}                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ {'gpt-4': 4.774774774774775, 'gpt-3.5-turbo': 4.774774774774775, 'gpt-4o': 4.818181818181818,               │ │
│ │ 'gpt-4o-mini': 4.818181818181818}                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Thought: The analysis has returned the average number of characters per token for each OpenAI model. Here are   │
│ the results:                                                                                                    │
│                                                                                                                 │
│  • GPT-4: 4.77 characters per token                                                                             │
│  • GPT-3.5-turbo: 4.77 characters per token                                                                     │
│  • GPT-4o: 4.82 characters per token                                                                            │
│  • GPT-4o-mini: 4.82 characters per token                                                                       │
│                                                                                                                 │
│ It appears that the averages are quite similar across the models, with only slight differences, particularly    │
│ between GPT-4 and GPT-4o models.                                                                                │
│                                                                                                                 │
│ Next, I will summarize these findings, providing a final analysis of how tokenization differs between the       │
│ studied models.                                                                                                 │
│                                                                                                                 │
│ FINAL ANSWER:                                                                                                   │
│                                                                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                    Average Characters per Token Analysis                                    ┃ │
│ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ │
│                                                                                                                 │
│                                                                                                                 │
│                                                     Results                                                     │
│                                                                                                                 │
│  • GPT-4: 4.77 characters per token                                                                             │
│  • GPT-3.5-turbo: 4.77 characters per token                                                                     │
│  • GPT-4o: 4.82 characters per token                                                                            │
│  • GPT-4o-mini: 4.82 characters per token                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│                                                     Summary                                                     │
│                                                                                                                 │
│ The analysis of tokenization across the different OpenAI models shows that the average number of characters per │
│ token is fairly consistent, typically around 4.77 to 4.82 characters. The slight variance suggests that while   │
│ tokenization strategies may differ slightly in handlin

### Summarize Some of My Blog Posts

#### gemini/gemini-2.0-flash-exp

In [36]:
task = """
I wrote a series of blog posts on my website here: https://drchrislevy.github.io/blog.html.
First generate a list of all the blog posts.
Pick the top 3 you think are most interesting and give me a one paragraph summary of each post.
Be sure to visit the the page of the actual blog posts you select to get the details for summarizing.
"""

In [37]:
messages_final = code_agent(task, model="gemini/gemini-2.0-flash-exp")

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ I wrote a series of blog posts on my website here: https://drchrislevy.github.io/blog.html.                     │
│ First generate a list of all the blog posts.                                                                    │
│ Pick the top 3 you think are most interesting and give me a one paragraph summary of each post.                 │
│ Be sure to visit the the page of the actual blog posts you select to get the details for summarizing.           │
│                                                                                                                 │
│                                                                                                                 │
╰─ gemini/gemini-2.0-flash-exp ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: The first step is to visit the provided blog page and extract all the blog post titles and links.      │
│ Then, I can determine the top 3 posts based on their titles or content. I'll look for engaging or               │
│ thought-provoking titles or themes among the posts.                                                             │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ visit_web_page("https://drchrislevy.github.io/blog.html")                                                       │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│   1 visit_web_page("https://drchrislevy.github.io/blog.html")                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Out[1]: 'Chris Levy - Blog\n\n[Chris Levy](./index.html)\n\n* [About](./index.html)\n*                      │ │
│ │ [Blog](./blog.html)\n\n \n \n\n# Blog\n\n                                                                   │ │
│ │ [![](./posts/modern_bert/imgs/intro.png)](./posts/modern_bert/modern_bert.html) \n\n[### Fine-Tuning        │ │
│ │ ModernBERT For Classification Tasks on Modal\n\nFirst go and read the ModernBert blog post announcement     │ │
│ │ here. If you are interested I wrote a little about transformers (encoders and decoders) in my previous blog │ │
│ │ posts here an…](./posts/modern_bert/modern_bert.html)\n\n[ ... <truncated>...                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ 'Chris Levy - Blog\n\n[Chris Levy](./index.html)\n\n* [About](./index.html)\n* [Blog](./blog.html)\n\n \n   │ │
│ │ \n\n# Blog\n\n [![](./posts/modern_bert/imgs/intro.png)](./posts/modern_bert/modern_bert.html) \n\n[###     │ │
│ │ Fine-Tuning ModernBERT For Classification Tasks on Modal\n\nFirst go and read the ModernBert blog post      │ │
│ │ announcement here. If you are interested I wrote a little about transformers (encoders and decoders) in my  │ │
│ │ previous blog posts here an…](./posts/modern_bert/modern_bert.html)\n\n[Dec 29,  ... <truncated>...         │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: I've accessed the content of the blog post "Memory Usage for Quantized LLMS." The post examines memory │
│ usage considerations particularly in the context of quantized models, focusing on inference and training. It    │
│ discusses quantization, its implications on memory and performance, and how to efficiently manage these aspects │
│ during model inference and training.                                                                            │
│                                                                                                                 │
│ Now, I will visit the third blog post, "Function Calling with Hermes-2-Pro-Mistral-7B," to gather details for   │
│ summarizing.                                                                                                    │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ visit_web_page("https://drchrislevy.github.io/posts/open_hermes_pro/open_hermes.html")                          │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│   1 visit_web_page("https://drchrislevy.github.io/posts/open_hermes_pro/open_hermes.html")                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Out[1]: 'Chris Levy - Function Calling with Hermes-2-Pro-Mistral-7B\n\n[Chris Levy](../../index.html)\n\n*  │ │
│ │ [About](../../index.html)\n* [Blog](../../blog.html)\n\n \n \n\n# Function Calling with                     │ │
│ │ Hermes-2-Pro-Mistral-7B\n\nAuthor\n\nChris Levy\n\nPublished\n\nMarch 16, 2024\n\nModified\n\nMarch 18,     │ │
│ │ 2024\n\n## On this page\n\n* [Introduction](#introduction)\n* [ENV Setup](#env-setup)\n* [LLM Inference     │ │
│ │ Class](#llm-inference-class)\n* [Function Calling Capabilities](#function-calling-capabilities)\n ...       │ │
│ │ <truncated>...                                                                                              │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ 'Chris Levy - Function Calling with Hermes-2-Pro-Mistral-7B\n\n[Chris Levy](../../index.html)\n\n*          │ │
│ │ [About](../../index.html)\n* [Blog](../../blog.html)\n\n \n \n\n# Function Calling with                     │ │
│ │ Hermes-2-Pro-Mistral-7B\n\nAuthor\n\nChris Levy\n\nPublished\n\nMarch 16, 2024\n\nModified\n\nMarch 18,     │ │
│ │ 2024\n\n## On this page\n\n* [Introduction](#introduction)\n* [ENV Setup](#env-setup)\n* [LLM Inference     │ │
│ │ Class](#llm-inference-class)\n* [Function Calling Capabilities](#function-calling-capabilities)\n  + [gpt   │ │
│ │ ... <truncated>...                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Here are the summaries for the top three blog posts:                                                            │
│                                                                                                                 │
│                           1. Fine-Tuning ModernBERT For Classification Tasks on Modal                           │
│                                                                                                                 │
│ This blog post discusses the process of fine-tuning the ModernBERT model to perform classification tasks        │
│ effectively using the Modal platform. The author provides step-by-step instructions on setting up the necessary │
│ environment, including creating a Modal account and setting up the environment for model training. Detailed     │
│ coding examples illustrate how to utilize the ModernBERT model, including how to encode text into embeddings,   │
│ manage datasets, and run model training. The post also covers the creation of a trainer file that facilitates   │
│ the training of the model with various datasets, such as emotion and AG News datasets. Overall, it serves as a  │
│ practical guide for developers looking to implement their own fine-tuning of NLP tasks using state-of-the-art   │
│ transformer models.                                                                                             │
│                                                                                                                 │
│                                       2. Memory Usage for Quantized LLMS                                        │
│                                                                                                                 │
│ In this post, Chris Levy outlines the memory considerations associated with large language models (LLMs),       │
│ especially when utilizing quantized models. The blog emphasizes the advantages of quantization in terms of      │
│ memory efficiency during model inference and training. High-level learnings from a recent LLM conference        │
│ provide insight into how bits and bytes relate to model performance, with practical examples showcasing how     │
│ different data types (like bfloat16 and float32) can impact memory usage. The post also offers demonstrations   │
│ using code snippets that analyze memory consumption for various quantized configurations, illustrating          │
│ potential memory savings while maintaining model effectiveness. It concludes with a discussion of trade-offs    │
│ associated with quantization and its implications for developers and researchers.                               │
│                                                                                                                 │
│                                3. Function Calling with Hermes-2-Pro-Mistral-7B                                 │
│                                                                                                                 │
│ This article explores the function calling capabilities of the open-source model                                │
│ NousResearch/Hermes-2-Pro-Mistral-7B, particularly in the context of enhanced interactivity in applications.    │
│ The author provides an introduction to setting up the programming environment with necessary dependencies like  │
│ OpenAI API, Langchain, and Hugging Face. Through code examples, the blog illustrates how to implement function  │
│ calling within the LLM system and how to convert Python functions into an OpenAI-compatible format. The         │
│ overview includes architecture for handling various tasks like querying weather forecasts, booking flights, and │
│ translating text via well-defined function protocols, empowering the LLM to interact seamlessly with external   │
│ applications. The post emphasizes the importance of do

### Download and Analyze Kaggle Dataset

#### claude-3-5-sonnet-20240620

In [38]:
task = """
Download the kaggle dataset: vijayveersingh/the-california-wildfire-data"
Perform some interesting analysis on the dataset and report on your findings.
I can not view plots yet so don't make plots. But please aggregate data and display as markdown tables.

You can download the dataset using the kagglehub library.
!pip install kagglehub
import kagglehub
path = kagglehub.dataset_download("vijayveersingh/the-california-wildfire-data"")
print("Path to dataset files:", path)
"""
messages_final = code_agent(task, model="claude-3-5-sonnet-20240620")

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ Download the kaggle dataset: vijayveersingh/the-california-wildfire-data"                                       │
│ Perform some interesting analysis on the dataset and report on your findings.                                   │
│ I can not view plots yet so don't make plots. But please aggregate data and display as markdown tables.         │
│                                                                                                                 │
│ You can download the dataset using the kagglehub library.                                                       │
│ !pip install kagglehub                                                                                          │
│ import kagglehub                                                                                                │
│ path = kagglehub.dataset_download("vijayveersingh/the-california-wildfire-data"")                               │
│ print("Path to dataset files:", path)                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─ claude-3-5-sonnet-20240620 ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: Let's begin by downloading the specified Kaggle dataset using the `kagglehub` library. We'll proceed   │
│ by ensuring the library is installed and then downloading the dataset to inspect its contents.                  │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ !pip install kagglehub                                                                                          │
│ import kagglehub                                                                                                │
│ path = kagglehub.dataset_download("vijayveersingh/the-california-wildfire-data")                                │
│ print("Path to dataset files:", path)                                                                           │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│   1 !pip install kagglehub                                                                                      │
│   2 import kagglehub                                                                                            │
│   3 path = kagglehub.dataset_download("vijayveersingh/the-california-wildfire-data")                            │
│   4 print("Path to dataset files:", path)                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Collecting kagglehub                                                                                        │ │
│ │   Downloading kagglehub-0.3.6-py3-none-any.whl.metadata (30 kB)                                             │ │
│ │ Collecting packaging (from kagglehub)                                                                       │ │
│ │   Downloading packaging-24.2-py3-none-any.whl.metadata (3.2 kB)                                             │ │
│ │ Requirement already satisfied: requests in /usr/local/lib/python3.11/site-packages (from kagglehub)         │ │
│ │ (2.32.3)                                                                                                    │ │
│ │ Collecting tqdm (from kagglehub)                                                                            │ │
│ │   Downloading tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)                                                 │ │
│ │      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--                                     │ │
│ │      ━━━━━━━━━━━━━━━━━━━━━━━━━━━ ... <truncated>...                                                         │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │   0%|          | 0.00/18.6M [00:00<?, ?B/s]                                                                 │ │
│ │  48%|####8     | 9.00M/18.6M [00:00<00:00, 92.1MB/s]                                                        │ │
│ │ 100%|##########| 18.6M/18.6M [00:00<00:00, 132MB/s]                                                         │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ None                                                                                                        │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Assistant Message ───────────────────────────────────────────────╮
│ Thought: It looks like the `tabulate` library, which is used for pretty-printing dataframes in markdown format, │
│ is not available in the environment. I will present the aggregated results of the analysis without using        │
│ `to_markdown`. Instead, I will format the results manually into markdown.                                       │
│                                                                                                                 │
│ Let's first aggregate the counts for damage severity and the top cities affected, then format them for display. │
│                                                                                                                 │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ # 1. Count the number of occurrences by damage severity                                                         │
│ damage_counts = wildfire_data['* Damage'].value_counts().reset_index()                                          │
│ damage_counts.columns = ['Damage Severity', 'Count']                                                            │
│                                                                                                                 │
│ # 2. Identify the top 5 cities affected by wildfires                                                            │
│ top_cities = wildfire_data['* City'].value_counts().head(5).reset_index()                                       │
│ top_cities.columns = ['City', 'Count']                                                                          │
│                                                                                                                 │
│ # Prepare markdown formatted outputs manually                                                                   │
│ damage_counts_markdown = "### Damage Severity Counts\n| Damage Severity | Count                                 │
│ |\n|-----------------|-------|\n"                                                                               │
│ for index, row in damage_counts.iterrows():                                                                     │
│     damage_counts_markdown += f"| {row['Damage Severity']} | {row['Count']} |\n"                                │
│                                                                                                                 │
│ top_cities_markdown = "### Top 5 Affected Cities\n| City            | Count |\n|-----------------|-------|\n"   │
│ for index, row in top_cities.iterrows():                                                                        │
│     top_cities_markdown += f"| {row['City']} | {row['Count']} |\n"                                              │
│                                                                                                                 │
│ # Display the markdown tables                                                                                   │
│ damage_counts_markdown, top_cities_markdown                                                                     │
│ ```<end_code>                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────  Executing Code ────────────────────────────────────────────────╮
│    1 # 1. Count the number of occurrences by damage severity                                                    │
│    2 damage_counts = wildfire_data['* Damage'].value_counts().reset_index()                                     │
│    3 damage_counts.columns = ['Damage Severity', 'Count']                                                       │
│    4                                                                                                            │
│    5 # 2. Identify the top 5 cities affected by wildfires                                                       │
│    6 top_cities = wildfire_data['* City'].value_counts().head(5).reset_index()                                  │
│    7 top_cities.columns = ['City', 'Count']                                                                     │
│    8                                                                                                            │
│    9 # Prepare markdown formatted outputs manually                                                              │
│   10 damage_counts_markdown = "### Damage Severity Counts\n| Damage Severity | Count                            │
│      |\n|-----------------|-------|\n"                                                                          │
│   11 for index, row in damage_counts.iterrows():                                                                │
│   12     damage_counts_markdown += f"| {row['Damage Severity']} | {row['Count']} |\n"                           │
│   13                                                                                                            │
│   14 top_cities_markdown = "### Top 5 Affected Cities\n| City            | Count                                │
│      |\n|-----------------|-------|\n"                                                                          │
│   15 for index, row in top_cities.iterrows():                                                                   │
│   16     top_cities_markdown += f"| {row['City']} | {row['Count']} |\n"                                         │
│   17                                                                                                            │
│   18 # Display the markdown tables                                                                              │
│   19 damage_counts_markdown, top_cities_markdown                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Code Execution Observation ───────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ Out[1]:                                                                                                     │ │
│ │ ('### Damage Severity Counts\n| Damage Severity | Count |\n|-----------------|-------|\n| Destroyed (>50%)  │ │
│ │ | 53879 |\n| No Damage | 40895 |\n| Affected (1-9%) | 3438 |\n| Minor (10-25%) | 1018 |\n| Major (26-50%) | │ │
│ │ 564 |\n| Inaccessible | 436 |\n',                                                                           │ │
│ │  '### Top 5 Affected Cities\n| City            | Count |\n|-----------------|-------|\n| Unincorporated |   │ │
│ │ 16201 |\n| Santa Rosa | 3278 |\n| Magalia | 2727 |\n| Paradise Northwest B | 1776 |\n| Paradise Central     │ │
│ │ Southwest B | 1574 |\n')                                                                                    │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── execution status ──────────────────────────────────────────────╮ │
│ │ Success: True                                                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── result ───────────────────────────────────────────────────╮ │
│ │ ('### Damage Severity Counts\n| Damage Severity | Count |\n|-----------------|-------|\n| Destroyed (>50%)  │ │
│ │ | 53879 |\n| No Damage | 40895 |\n| Affected (1-9%) | 3438 |\n| Minor (10-25%) | 1018 |\n| Major (26-50%) | │ │
│ │ 564 |\n| Inaccessible | 436 |\n', '### Top 5 Affected Cities\n| City            | Count                     │ │
│ │ |\n|-----------------|-------|\n| Unincorporated | 16201 |\n| Santa Rosa | 3278 |\n| Magalia | 2727 |\n|    │ │
│ │ Paradise Northwest B | 1776 |\n| Paradise Central Southwest B | 1574 |\n')                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ FINAL ANSWER: Here are the findings from the analysis of the California wildfire dataset:                       │
│                                                                                                                 │
│                                             Damage Severity Counts                                              │
│                                                                                                                 │
│                                                                                                                 │
│   Damage Severity    Count                                                                                      │
│  ━━━━━━━━━━━━━━━━━━━━━━━━━━━                                                                                    │
│   Destroyed (>50%)   53,879                                                                                     │
│   No Damage          40,895                                                                                     │
│   Affected (1-9%)    3,438                                                                                      │
│   Minor (10-25%)     1,018                                                                                      │
│   Major (26-50%)     564                                                                                        │
│   Inaccessible       436                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│                                              Top 5 Affected Cities                                              │
│                                                                                                                 │
│                                                                                                                 │
│   City                           Count                                                                          │
│  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━                                                                        │
│   Unincorporated                 16,201                                                                         │
│   Santa Rosa                     3,278                                                                          │
│   Magalia                        2,727                                                                          │
│   Paradise Northwest B           1,776                                                                          │
│   Paradise Central Southwest B   1,574                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│ These tables summarize the overall damage sustained during the wildfires and the cities that were most          │
│ affected.                                                                                                       │
╰─ final-answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

# Part 2 (TBD)

This exploration of AI agents has just scratched the surface. I planned to do much more but sort of ran out of steam.
I'm going to come back to this in the future.
This intro was all about skipping the frameworks and just playing around with tools and loops and seeing what was out there.
I found `smolagents` library to be a great learning resource. They just announced a new version that supports vision capabilities so more to learn there.

There is so much more I read and minimally investigated, but I need more time to dig more into things. I tried to keep the list of resources 
I was reading and also ones I want to explore in the future. They are below and in no particular order.

# Resources (Random Order)

[smolagents can see](https://huggingface.co/blog/smolagents-can-see)

[OpenAI Note Book - Prerequisite to Swarm - Orchestrating Agents: Routines and Handoffs](https://github.com/openai/openai-cookbook/blob/main/examples/Orchestrating_agents.ipynb)

[Anthropic Blog - Building effective agents](https://docs.anthropic.com/en/docs/build/agents/building-effective-agentshttps://www.anthropic.com/research/building-effective-agents)

[AI Engineering Book - By Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/)

[Hugging Face Blog Post - Introducing smolagents, a simple library to build agents](https://huggingface.co/blog/smolagents)

[litelllm - GIthub](https://github.com/BerriAI/litellm)


[Hugging Face Collection of Papers on Agents](https://huggingface.co/collections/m-ric/agents-65ba776fbd9e29f771c07d4e)

[Deep Learning AI Course -  Multi AI Agent Systems with crewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)

[Raising the bar on SWE-bench Verified with Claude 3.5 Sonnet](https://www.anthropic.com/research/swe-bench-sonnet)

[Building Effective Agents Cookbook](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)

[deeplearning.ai - The Batch - Issue 281 - Recap of 2024 - with some agentic thoughts](https://www.deeplearning.ai/the-batch/issue-281/)

[LangChain Blog Post - What is an Agent?](https://blog.langchain.dev/what-is-an-agent/)

[Lindy](https://www.lindy.ai/)


[AWS: What is an Agent?](https://aws.amazon.com/what-is/ai-agents/)

[Chapter 6 Agents from AI Engineering Book by Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

[tweet from Andrew Ng on AI Agent Spectrum](https://x.com/AndrewYNg/status/1801295202788983136)

[Nathan Lambert Blog Post on the AI Agent Spectrum](https://www.interconnects.ai/p/the-ai-agent-spectrum)

[Lang Chain Academy - AI Agents with LangGraph](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

[Deep Learning AI Course - LangGraph AI Agents](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/)

[A simple Python implementation of the ReAct pattern for LLMs](https://til.simonwillison.net/llms/python-react-pattern)

[AI Agents That Matter](https://agents.cs.princeton.edu/)

[Demystifying AI Agents: A Guide for Beginners](https://www.mongodb.com/resources/basics/artificial-intelligence/ai-agents)

[REACT: SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS](https://arxiv.org/pdf/2210.03629)

[Chip Huyen Blog post on AI Agents](https://x.com/chipro/status/1876681640505901266)

[321 real-world gen AI use cases from the world's leading organizations](https://cloud.google.com/transform/101-real-world-generative-ai-use-cases-from-industry-leaders)

[LLM Agents MOOC YouTube Playlist by Berkeley RDI Center on Decentralization & AI](https://www.youtube.com/playlist?list=PLS01nW3RtgopsNLeM936V4TNSsvvVglLc)

[Which AI Agent framework should i use? (CrewAI, Langgraph, Majestic-one and pure code)](https://medium.com/@aydinKerem/which-ai-agent-framework-i-should-use-crewai-langgraph-majestic-one-and-pure-code-e16a6e4d9252)

[crewAI](https://github.com/crewAIInc/crewAI)

[Langgraph](https://github.com/langchain-ai/langgraph)

[Smolagents](https://github.com/huggingface/smolagents)

[autogen](https://github.com/microsoft/autogen)

[Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/)

[Arcade: Natural Language to Code Generation in Interactive Computing Notebooks](https://github.com/google-research/arcade-nl2code)

[Executable Code Actions Elicit Better LLM Agents](https://github.com/xingyaoww/code-act)

[SWE-bench](https://www.swebench.com/)

[SWE-agent](https://github.com/SWE-agent/SWE-agent)

[OpenHands](https://github.com/All-Hands-AI/OpenHands)

[aider](https://aider.chat/)

[TapeAgents](https://github.com/ServiceNow/TapeAgents)

[DynaSaur: Large Language Agents Beyond Predefined Actions](https://huggingface.co/papers/2411.01747)